In [1]:
# TFM_03_objectives_and_plots.ipynb

In [2]:

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
import warnings
import logging
import time
import os

pd.set_option('display.float_format', lambda x: '{:.12f}'.format(x))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

tqdm.pandas()

In [3]:
file_path = 'indicators/df_full.csv'

EXPECTED_MAX_LAG = 30
REQUIRED_CORE_COLS = ['timestamp', 'symbol', 'close', 'volume']
LAG_BASES = ['close', 'high', 'low']

logging.info(f"Cargando archivo: {file_path}")

try:
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    logging.info(f"Archivo cargado. Shape: {df.shape}")

    actual_columns = set(df.columns)
    missing_core_cols = [col for col in REQUIRED_CORE_COLS if col not in actual_columns]
    if missing_core_cols:
        raise ValueError(f"Columnas requeridas faltantes: {missing_core_cols}")
    logging.info("Verificación de columnas core completada.")

    df = df.sort_values(by=['symbol', 'timestamp']).reset_index(drop=True)
    logging.info("DataFrame ordenado por symbol y timestamp.")

    expected_lag_cols = {
        f'{base}_lag_{i}'
        for i in range(EXPECTED_MAX_LAG + 1)
        for base in LAG_BASES
    }
    missing_lag_cols = sorted(list(expected_lag_cols - actual_columns))

    if missing_lag_cols:
        logging.warning(f"Faltan {len(missing_lag_cols)} columnas de lag esperadas (hasta lag {EXPECTED_MAX_LAG}).")
        logging.warning(f"Primeras 10 columnas faltantes: {missing_lag_cols[:10]}")

    print("\nInformación del DataFrame:")
    df.info()

except FileNotFoundError:
    logging.error(f"Archivo no encontrado: '{file_path}'.")
    raise
except ValueError as ve:
    logging.error(f"Error en validación de datos: {ve}")
    raise

2025-04-06 13:05:20,122 - INFO - Cargando archivo: indicators/df_full.csv
2025-04-06 13:06:22,244 - INFO - Archivo cargado. Shape: (432450, 486)
2025-04-06 13:06:22,336 - INFO - Verificación de columnas core completada.
2025-04-06 13:06:26,383 - INFO - DataFrame ordenado por symbol y timestamp.



Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432450 entries, 0 to 432449
Columns: 486 entries, timestamp to MFI_20
dtypes: bool(13), float64(471), object(2)
memory usage: 1.5+ GB


In [4]:
df.sample(5)

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_lag_0,volume_lag_0,open_lag_0,high_lag_0,low_lag_0,quote_asset_volume_lag_0,number_of_trades_lag_0,taker_buy_base_asset_volume_lag_0,taker_buy_quote_asset_volume_lag_0,close_lag_1,volume_lag_1,open_lag_1,high_lag_1,low_lag_1,quote_asset_volume_lag_1,number_of_trades_lag_1,taker_buy_base_asset_volume_lag_1,taker_buy_quote_asset_volume_lag_1,close_lag_2,volume_lag_2,open_lag_2,high_lag_2,low_lag_2,quote_asset_volume_lag_2,number_of_trades_lag_2,taker_buy_base_asset_volume_lag_2,taker_buy_quote_asset_volume_lag_2,close_lag_3,volume_lag_3,open_lag_3,high_lag_3,low_lag_3,quote_asset_volume_lag_3,number_of_trades_lag_3,taker_buy_base_asset_volume_lag_3,taker_buy_quote_asset_volume_lag_3,close_lag_4,volume_lag_4,open_lag_4,high_lag_4,low_lag_4,quote_asset_volume_lag_4,number_of_trades_lag_4,taker_buy_base_asset_volume_lag_4,taker_buy_quote_asset_volume_lag_4,close_lag_5,volume_lag_5,open_lag_5,high_lag_5,low_lag_5,quote_asset_volume_lag_5,number_of_trades_lag_5,taker_buy_base_asset_volume_lag_5,taker_buy_quote_asset_volume_lag_5,close_lag_6,volume_lag_6,open_lag_6,high_lag_6,low_lag_6,quote_asset_volume_lag_6,number_of_trades_lag_6,taker_buy_base_asset_volume_lag_6,taker_buy_quote_asset_volume_lag_6,close_lag_7,volume_lag_7,open_lag_7,high_lag_7,low_lag_7,quote_asset_volume_lag_7,number_of_trades_lag_7,taker_buy_base_asset_volume_lag_7,taker_buy_quote_asset_volume_lag_7,close_lag_8,volume_lag_8,open_lag_8,high_lag_8,low_lag_8,quote_asset_volume_lag_8,number_of_trades_lag_8,taker_buy_base_asset_volume_lag_8,taker_buy_quote_asset_volume_lag_8,close_lag_9,volume_lag_9,open_lag_9,high_lag_9,low_lag_9,quote_asset_volume_lag_9,number_of_trades_lag_9,taker_buy_base_asset_volume_lag_9,taker_buy_quote_asset_volume_lag_9,close_lag_10,volume_lag_10,open_lag_10,high_lag_10,low_lag_10,quote_asset_volume_lag_10,number_of_trades_lag_10,taker_buy_base_asset_volume_lag_10,taker_buy_quote_asset_volume_lag_10,close_lag_11,volume_lag_11,open_lag_11,high_lag_11,low_lag_11,quote_asset_volume_lag_11,number_of_trades_lag_11,taker_buy_base_asset_volume_lag_11,taker_buy_quote_asset_volume_lag_11,close_lag_12,volume_lag_12,open_lag_12,high_lag_12,low_lag_12,quote_asset_volume_lag_12,number_of_trades_lag_12,taker_buy_base_asset_volume_lag_12,taker_buy_quote_asset_volume_lag_12,close_lag_13,volume_lag_13,open_lag_13,high_lag_13,low_lag_13,quote_asset_volume_lag_13,number_of_trades_lag_13,taker_buy_base_asset_volume_lag_13,taker_buy_quote_asset_volume_lag_13,close_lag_14,volume_lag_14,open_lag_14,high_lag_14,low_lag_14,quote_asset_volume_lag_14,number_of_trades_lag_14,taker_buy_base_asset_volume_lag_14,taker_buy_quote_asset_volume_lag_14,close_lag_15,volume_lag_15,open_lag_15,high_lag_15,low_lag_15,quote_asset_volume_lag_15,number_of_trades_lag_15,taker_buy_base_asset_volume_lag_15,taker_buy_quote_asset_volume_lag_15,close_lag_16,volume_lag_16,open_lag_16,high_lag_16,low_lag_16,quote_asset_volume_lag_16,number_of_trades_lag_16,taker_buy_base_asset_volume_lag_16,taker_buy_quote_asset_volume_lag_16,close_lag_17,volume_lag_17,open_lag_17,high_lag_17,low_lag_17,quote_asset_volume_lag_17,number_of_trades_lag_17,taker_buy_base_asset_volume_lag_17,taker_buy_quote_asset_volume_lag_17,close_lag_18,volume_lag_18,open_lag_18,high_lag_18,low_lag_18,quote_asset_volume_lag_18,number_of_trades_lag_18,taker_buy_base_asset_volume_lag_18,taker_buy_quote_asset_volume_lag_18,close_lag_19,volume_lag_19,open_lag_19,high_lag_19,low_lag_19,quote_asset_volume_lag_19,number_of_trades_lag_19,taker_buy_base_asset_volume_lag_19,taker_buy_quote_asset_volume_lag_19,close_lag_20,volume_lag_20,open_lag_20,high_lag_20,low_lag_20,quote_asset_volume_lag_20,number_of_trades_lag_20,taker_buy_base_asset_volume_lag_20,taker_buy_quote_asset_volume_lag_20,close_lag_21,volume_lag_21,open_lag_21,high_lag_21,low_lag_21,quote_asset_volume_lag_21,number_of_trades_lag_

In [5]:
PAST_LAGS = 30
FUTURE_LEADS = 15

MIN_DURATION = 5
MAX_DURATION = 28
MIN_FUTURE_INCREASE_PERCENT = 1.5 / 100
START_LEAD_FOR_CHANGE = 5
END_LEAD_FOR_CHANGE = 15
CAP_LIMIT = 1.0
MAX_RELATIVE_DISTANCE = 0.4

logging.info("Parámetros de análisis definidos.")
logging.info(f"PAST_LAGS={PAST_LAGS}, FUTURE_LEADS={FUTURE_LEADS}")
logging.info(f"MIN_DURATION={MIN_DURATION}, MAX_DURATION={MAX_DURATION}")
logging.info(f"MIN_FUTURE_INCREASE_PERCENT={MIN_FUTURE_INCREASE_PERCENT:.4f}")
logging.info(f"CAP_LIMIT={CAP_LIMIT}, MAX_RELATIVE_DISTANCE={MAX_RELATIVE_DISTANCE}")

2025-04-06 13:06:27,199 - INFO - Parámetros de análisis definidos.
2025-04-06 13:06:27,201 - INFO - PAST_LAGS=30, FUTURE_LEADS=15
2025-04-06 13:06:27,203 - INFO - MIN_DURATION=5, MAX_DURATION=28
2025-04-06 13:06:27,204 - INFO - MIN_FUTURE_INCREASE_PERCENT=0.0150
2025-04-06 13:06:27,205 - INFO - CAP_LIMIT=1.0, MAX_RELATIVE_DISTANCE=0.4


In [6]:
logging.info("Calculando 'future_max_increase_capped'...")
tiempo_inicio_calculo = time.time()

columnas_lead = [f'close_lead_{i}' for i in range(START_LEAD_FOR_CHANGE, END_LEAD_FOR_CHANGE + 1)]
columna_precio_base = 'close_lag_0'

columnas_a_usar = columnas_lead + [columna_precio_base]
datos_calculo = df[columnas_a_usar].apply(pd.to_numeric, errors='coerce')

precio_base = datos_calculo[columna_precio_base]
maximo_futuro = datos_calculo[columnas_lead].max(axis=1, skipna=True)

base_segura = precio_base.where(precio_base > 1e-9)
incremento_porcentual = (maximo_futuro - precio_base) / base_segura

incremento_futuro_limitado_con_nan = np.clip(incremento_porcentual, a_min=0, a_max=CAP_LIMIT)
incremento_futuro_final = incremento_futuro_limitado_con_nan.fillna(0)

df['future_max_increase_capped'] = incremento_futuro_final

logging.info(f"Cálculo de 'future_max_increase_capped' completado en {time.time() - tiempo_inicio_calculo:.2f} seg.")
print("\nDescripción 'future_max_increase_capped' (%):")
print((df['future_max_increase_capped'] * 100).describe())

2025-04-06 13:06:27,224 - INFO - Calculando 'future_max_increase_capped'...
2025-04-06 13:06:27,813 - INFO - Cálculo de 'future_max_increase_capped' completado en 0.59 seg.



Descripción 'future_max_increase_capped' (%):
count   432450.000000000000
mean         0.380676617284
std          0.512682008940
min          0.000000000000
25%          0.000000000000
50%          0.238663484487
75%          0.571428571429
max         22.666666666667
Name: future_max_increase_capped, dtype: float64


In [7]:
def detect_valley_shape_and_target(row, past_lags, future_leads,
                                     max_relative_distance,
                                     min_duration, max_duration,
                                     min_future_increase_perc,
                                     start_lead_for_increase):
    """
    Detecta una forma de valle sinusoidal y determina si cumple el target de subida futura.

    Args:
        row (pd.Series): Fila del DataFrame con datos de lags y leads.
        past_lags (int): Número de lags pasados a considerar.
        future_leads (int): Número de leads futuros a considerar.
        max_relative_distance (float): Distancia relativa máxima permitida al ajuste ideal.
        min_duration (int): Duración mínima del valle (en minutos/periodos).
        max_duration (int): Duración máxima del valle (en minutos/periodos).
        min_future_increase_perc (float): Porcentaje mínimo de subida futura requerido.
        start_lead_for_increase (int): Lead inicial para calcular la subida futura.

    Returns:
        pd.Series: Resultados del análisis (target, is_sinusoid_shape, etc.).
    """
    PORCENTAJE_MINIMO_DENTRO_DISTANCIA = 0.80
    PORCENTAJE_MINIMO_AMPLITUD_RELATIVA = 0.015

    resultado_inicial = {
        'target': 0,
        'is_sinusoid_shape': 0,
        'valley_price_at_0': 0.0,
        'duration_minutes': 0,
        'mse_fit': 0.0,
        'future_increase_perc_calc': 0.0,
        'preceding_peak_lag': -1,
        'preceding_peak_price': 0.0,
        'actual_prices_fit': [],
        'ideal_curve_fit': []
    }
    forma_detectada = False

    try:
        cierre_actual = row.get(f'close_lag_0')
        minimo_actual = row.get(f'low_lag_0', cierre_actual)

        if pd.isna(cierre_actual) or pd.isna(minimo_actual) or cierre_actual <= 1e-9:
            return pd.Series(resultado_inicial)
        resultado_inicial['valley_price_at_0'] = cierre_actual

        minimos_pasados = [row.get(f'low_lag_{i}', np.inf) for i in range(1, past_lags + 1)]
        if not minimos_pasados or not (minimo_actual <= min(minimos_pasados)):
             return pd.Series(resultado_inicial)

        maximos_lags = {i: row.get(f'high_lag_{i}') for i in range(1, past_lags + 1)}
        maximos_lags_validos = {
            lag: price for lag, price in maximos_lags.items()
            if pd.notna(price) and price > minimo_actual
        }
        if not maximos_lags_validos:
            return pd.Series(resultado_inicial)

        lag_pico_precedente = max(maximos_lags_validos, key=maximos_lags_validos.get)
        precio_pico_precedente = maximos_lags_validos[lag_pico_precedente]

        diferencia_pico_valle = precio_pico_precedente - cierre_actual
        amplitud_relativa = diferencia_pico_valle / cierre_actual
        if amplitud_relativa < PORCENTAJE_MINIMO_AMPLITUD_RELATIVA:
             return pd.Series(resultado_inicial)

        resultado_inicial['preceding_peak_lag'] = lag_pico_precedente
        resultado_inicial['preceding_peak_price'] = precio_pico_precedente

        duracion = lag_pico_precedente
        if not (min_duration <= duracion <= max_duration):
            return pd.Series(resultado_inicial)
        resultado_inicial['duration_minutes'] = duracion

        precios_reales = [row.get(f'close_lag_{i}') for i in range(lag_pico_precedente, -1, -1)]
        if any(pd.isna(p) for p in precios_reales) or len(precios_reales) <= 2:
            return pd.Series(resultado_inicial)

        precios_reales_np = np.array(precios_reales)
        numero_puntos = len(precios_reales_np)

        amplitud = diferencia_pico_valle / 2
        punto_medio = (precio_pico_precedente + cierre_actual) / 2

        if amplitud <= 1e-9:
             return pd.Series(resultado_inicial)
        curva_ideal = amplitud * np.cos(np.pi * np.linspace(0, 1, numero_puntos)) + punto_medio

        distancias_verticales = np.abs(precios_reales_np - curva_ideal)
        distancias_intermedias = distancias_verticales[1:-1]
        distancia_maxima_permitida = max_relative_distance * diferencia_pico_valle

        if len(distancias_intermedias) > 0:
            puntos_cercanos = np.sum(distancias_intermedias <= distancia_maxima_permitida)
            porcentaje_cercano = puntos_cercanos / len(distancias_intermedias)
        else:
            porcentaje_cercano = 1.0

        if porcentaje_cercano < PORCENTAJE_MINIMO_DENTRO_DISTANCIA:
             return pd.Series(resultado_inicial)

        resultado_inicial['is_sinusoid_shape'] = 1
        forma_detectada = True
        resultado_inicial['actual_prices_fit'] = list(precios_reales_np)
        resultado_inicial['ideal_curve_fit'] = list(curva_ideal)

        try:
            error_cuadratico_medio = np.mean((precios_reales_np - curva_ideal)**2)
            resultado_inicial['mse_fit'] = error_cuadratico_medio
        except Exception:
             resultado_inicial['mse_fit'] = 0.0

        precios_cierre_futuros = [row.get(f'close_lead_{i}') for i in range(start_lead_for_increase, future_leads + 1)]
        precios_futuros_validos_numericos = [pd.to_numeric(p, errors='coerce') for p in precios_cierre_futuros]
        precios_futuros_validos = [p for p in precios_futuros_validos_numericos if pd.notna(p)]

        if not precios_futuros_validos:
             return pd.Series(resultado_inicial)

        maximo_precio_futuro = max(precios_futuros_validos)
        incremento_futuro_porcentual = (maximo_precio_futuro - cierre_actual) / cierre_actual
        resultado_inicial['future_increase_perc_calc'] = incremento_futuro_porcentual

        if incremento_futuro_porcentual >= min_future_increase_perc:
            resultado_inicial['target'] = 1

    except Exception:
        pass

    if not forma_detectada:
        resultado_inicial['actual_prices_fit'] = []
        resultado_inicial['ideal_curve_fit'] = []

    return pd.Series(resultado_inicial)

logging.info("Función 'detect_valley_shape_and_target' definida.")

2025-04-06 13:06:27,950 - INFO - Función 'detect_valley_shape_and_target' definida.


In [8]:
logging.info(f"Aplicando 'detect_valley_shape_and_target' a {len(df):,} filas...")
tiempo_inicio_apply = time.time()

resultados_df = df.progress_apply(
    lambda fila: detect_valley_shape_and_target(
        fila,
        past_lags=PAST_LAGS,
        future_leads=FUTURE_LEADS,
        max_relative_distance=MAX_RELATIVE_DISTANCE,
        min_duration=MIN_DURATION,
        max_duration=MAX_DURATION,
        min_future_increase_perc=MIN_FUTURE_INCREASE_PERCENT,
        start_lead_for_increase=START_LEAD_FOR_CHANGE
    ),
    axis=1,
    result_type='expand'
)

tiempo_fin_apply = time.time()
logging.info(f"Procesamiento completado en {tiempo_fin_apply - tiempo_inicio_apply:.2f} segundos.")

df_procesado = df.join(resultados_df)

logging.info(f"Columnas de análisis añadidas. Nueva shape: {df_procesado.shape}")

conteo_target_1 = df_procesado['target'].sum()
conteo_forma_detectada = df_procesado['is_sinusoid_shape'].sum()
print(f"\nTarget=1 encontrados: {conteo_target_1:,}")
print(f"Formas detectadas (is_sinusoid_shape=1): {conteo_forma_detectada:,}")

logging.info("Calculando timestamps de picos precedentes...")

if not pd.api.types.is_datetime64_any_dtype(df_procesado['timestamp']):
    df_procesado['timestamp'] = pd.to_datetime(df_procesado['timestamp'], errors='coerce')
df_procesado['preceding_peak_lag'] = pd.to_numeric(df_procesado['preceding_peak_lag'], errors='coerce')

df_procesado['preceding_peak_ts'] = pd.NaT

mascara_pico_valido = (
    df_procesado['preceding_peak_lag'].notna()
    & (df_procesado['preceding_peak_lag'] != -1)
    & df_procesado['timestamp'].notna()
)

if mascara_pico_valido.any():
    indices_validos = df_procesado.index[mascara_pico_valido]
    lags_para_calculo = df_procesado.loc[indices_validos, 'preceding_peak_lag']
    timestamps_base = df_procesado.loc[indices_validos, 'timestamp']

    deltas_tiempo = pd.to_timedelta(lags_para_calculo, unit='m', errors='coerce')

    mascara_delta_valido = deltas_tiempo.notna()
    indices_finales = indices_validos[mascara_delta_valido]

    if not indices_finales.empty:
        df_procesado.loc[indices_finales, 'preceding_peak_ts'] = \
            timestamps_base.loc[indices_finales] - deltas_tiempo.loc[indices_finales]

logging.info("Timestamps precedentes calculados.")
print("\nDataFrame 'df_procesado' actualizado.")

logging.info("Comparando conteos: Target vs Subida futura mínima...")
print(f"Filas con Target = 1: {conteo_target_1:,}")
conteo_subida_futura_minima = (df_procesado['future_max_increase_capped'] >= MIN_FUTURE_INCREASE_PERCENT).sum()
print(f"Filas con Subida Futura >= {MIN_FUTURE_INCREASE_PERCENT*100:.1f}%: {conteo_subida_futura_minima:,}")

if conteo_target_1 > 0 and conteo_subida_futura_minima > 0:
    porcentaje_target_vs_subida = (conteo_target_1 / conteo_subida_futura_minima) * 100
    print(f"Target=1 representa el {porcentaje_target_vs_subida:.2f}% de las filas con subida futura mínima.")
elif conteo_subida_futura_minima == 0:
     print("No hubo filas con la subida futura mínima requerida.")
else:
     print("No se puede calcular el porcentaje comparativo (target=0).")

2025-04-06 13:06:27,983 - INFO - Aplicando 'detect_valley_shape_and_target' a 432,450 filas...


  0%|          | 0/432450 [00:00<?, ?it/s]

2025-04-06 13:10:28,260 - INFO - Procesamiento completado en 240.28 segundos.
2025-04-06 13:10:34,328 - INFO - Columnas de análisis añadidas. Nueva shape: (432450, 497)
2025-04-06 13:10:34,350 - INFO - Calculando timestamps de picos precedentes...



Target=1 encontrados: 171
Formas detectadas (is_sinusoid_shape=1): 10,171


2025-04-06 13:10:34,710 - INFO - Timestamps precedentes calculados.
2025-04-06 13:10:34,714 - INFO - Comparando conteos: Target vs Subida futura mínima...



DataFrame 'df_procesado' actualizado.
Filas con Target = 1: 171
Filas con Subida Futura >= 1.5%: 13,534
Target=1 representa el 1.26% de las filas con subida futura mínima.


In [9]:
df_procesado.sample(5)

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_lag_0,volume_lag_0,open_lag_0,high_lag_0,low_lag_0,quote_asset_volume_lag_0,number_of_trades_lag_0,taker_buy_base_asset_volume_lag_0,taker_buy_quote_asset_volume_lag_0,close_lag_1,volume_lag_1,open_lag_1,high_lag_1,low_lag_1,quote_asset_volume_lag_1,number_of_trades_lag_1,taker_buy_base_asset_volume_lag_1,taker_buy_quote_asset_volume_lag_1,close_lag_2,volume_lag_2,open_lag_2,high_lag_2,low_lag_2,quote_asset_volume_lag_2,number_of_trades_lag_2,taker_buy_base_asset_volume_lag_2,taker_buy_quote_asset_volume_lag_2,close_lag_3,volume_lag_3,open_lag_3,high_lag_3,low_lag_3,quote_asset_volume_lag_3,number_of_trades_lag_3,taker_buy_base_asset_volume_lag_3,taker_buy_quote_asset_volume_lag_3,close_lag_4,volume_lag_4,open_lag_4,high_lag_4,low_lag_4,quote_asset_volume_lag_4,number_of_trades_lag_4,taker_buy_base_asset_volume_lag_4,taker_buy_quote_asset_volume_lag_4,close_lag_5,volume_lag_5,open_lag_5,high_lag_5,low_lag_5,quote_asset_volume_lag_5,number_of_trades_lag_5,taker_buy_base_asset_volume_lag_5,taker_buy_quote_asset_volume_lag_5,close_lag_6,volume_lag_6,open_lag_6,high_lag_6,low_lag_6,quote_asset_volume_lag_6,number_of_trades_lag_6,taker_buy_base_asset_volume_lag_6,taker_buy_quote_asset_volume_lag_6,close_lag_7,volume_lag_7,open_lag_7,high_lag_7,low_lag_7,quote_asset_volume_lag_7,number_of_trades_lag_7,taker_buy_base_asset_volume_lag_7,taker_buy_quote_asset_volume_lag_7,close_lag_8,volume_lag_8,open_lag_8,high_lag_8,low_lag_8,quote_asset_volume_lag_8,number_of_trades_lag_8,taker_buy_base_asset_volume_lag_8,taker_buy_quote_asset_volume_lag_8,close_lag_9,volume_lag_9,open_lag_9,high_lag_9,low_lag_9,quote_asset_volume_lag_9,number_of_trades_lag_9,taker_buy_base_asset_volume_lag_9,taker_buy_quote_asset_volume_lag_9,close_lag_10,volume_lag_10,open_lag_10,high_lag_10,low_lag_10,quote_asset_volume_lag_10,number_of_trades_lag_10,taker_buy_base_asset_volume_lag_10,taker_buy_quote_asset_volume_lag_10,close_lag_11,volume_lag_11,open_lag_11,high_lag_11,low_lag_11,quote_asset_volume_lag_11,number_of_trades_lag_11,taker_buy_base_asset_volume_lag_11,taker_buy_quote_asset_volume_lag_11,close_lag_12,volume_lag_12,open_lag_12,high_lag_12,low_lag_12,quote_asset_volume_lag_12,number_of_trades_lag_12,taker_buy_base_asset_volume_lag_12,taker_buy_quote_asset_volume_lag_12,close_lag_13,volume_lag_13,open_lag_13,high_lag_13,low_lag_13,quote_asset_volume_lag_13,number_of_trades_lag_13,taker_buy_base_asset_volume_lag_13,taker_buy_quote_asset_volume_lag_13,close_lag_14,volume_lag_14,open_lag_14,high_lag_14,low_lag_14,quote_asset_volume_lag_14,number_of_trades_lag_14,taker_buy_base_asset_volume_lag_14,taker_buy_quote_asset_volume_lag_14,close_lag_15,volume_lag_15,open_lag_15,high_lag_15,low_lag_15,quote_asset_volume_lag_15,number_of_trades_lag_15,taker_buy_base_asset_volume_lag_15,taker_buy_quote_asset_volume_lag_15,close_lag_16,volume_lag_16,open_lag_16,high_lag_16,low_lag_16,quote_asset_volume_lag_16,number_of_trades_lag_16,taker_buy_base_asset_volume_lag_16,taker_buy_quote_asset_volume_lag_16,close_lag_17,volume_lag_17,open_lag_17,high_lag_17,low_lag_17,quote_asset_volume_lag_17,number_of_trades_lag_17,taker_buy_base_asset_volume_lag_17,taker_buy_quote_asset_volume_lag_17,close_lag_18,volume_lag_18,open_lag_18,high_lag_18,low_lag_18,quote_asset_volume_lag_18,number_of_trades_lag_18,taker_buy_base_asset_volume_lag_18,taker_buy_quote_asset_volume_lag_18,close_lag_19,volume_lag_19,open_lag_19,high_lag_19,low_lag_19,quote_asset_volume_lag_19,number_of_trades_lag_19,taker_buy_base_asset_volume_lag_19,taker_buy_quote_asset_volume_lag_19,close_lag_20,volume_lag_20,open_lag_20,high_lag_20,low_lag_20,quote_asset_volume_lag_20,number_of_trades_lag_20,taker_buy_base_asset_volume_lag_20,taker_buy_quote_asset_volume_lag_20,close_lag_21,volume_lag_21,open_lag_21,high_lag_21,low_lag_21,quote_asset_volume_lag_21,number_of_trades_lag_

In [10]:
def plot_confirmed_valleys_detailed(df_processed, symbol, date_str=""):
    """
    Genera y guarda un gráfico detallado de los valles confirmados (target=1) para un símbolo.

    Args:
        df_processed (pd.DataFrame): DataFrame con los resultados del análisis.
        symbol (str): Símbolo a graficar.
        date_str (str): Etiqueta de fecha opcional para el título y nombre de archivo.

    Returns:
        None: La función muestra y guarda la gráfica.
    """
    logging.info(f"Generando gráfico detallado (target=1) para: {symbol}")

    df_simbolo = df_processed[df_processed['symbol'] == symbol].copy()
    if df_simbolo.empty:
        logging.warning(f"No hay datos para '{symbol}'.")
        return

    if not pd.api.types.is_datetime64_any_dtype(df_simbolo['timestamp']):
        df_simbolo['timestamp'] = pd.to_datetime(df_simbolo['timestamp'], errors='coerce')
        df_simbolo.dropna(subset=['timestamp'], inplace=True)
        if df_simbolo.empty:
            logging.warning(f"No quedan datos válidos para '{symbol}' tras conversión.")
            return

    valles_confirmados = df_simbolo[df_simbolo['target'] == 1].copy()
    if valles_confirmados.empty:
        logging.info(f"No se encontraron puntos target=1 para '{symbol}'.")
        return

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03,
                        row_heights=[0.8, 0.2],
                        subplot_titles=(f"{symbol} - Valles Confirmados (Target=1)", "Volumen"))

    columna_precio_base = 'close_lag_0'

    fig.add_trace(go.Scatter(x=df_simbolo['timestamp'], y=df_simbolo[columna_precio_base],
                             mode='lines', line=dict(color='darkgrey', width=1),
                             name='Precio', legendgroup='price'), row=1, col=1)

    leyendas_mostradas = set()
    logging.info(f"Graficando {len(valles_confirmados)} valles confirmados para {symbol}...")

    for indice, fila in valles_confirmados.iterrows():
        precios_reales = fila.get('actual_prices_fit', [])
        curva_ideal = fila.get('ideal_curve_fit', [])
        ts_inicio = fila.get('preceding_peak_ts')
        ts_fin = fila['timestamp']

        if isinstance(precios_reales, (list, np.ndarray)) and len(precios_reales) > 1 and pd.notna(ts_inicio) and pd.notna(ts_fin):
            duracion_puntos = len(precios_reales)
            try:
                puntos_tiempo = pd.date_range(ts_inicio, ts_fin, periods=duracion_puntos, name=f'fit_{indice}')
            except ValueError as err:
                logging.warning(f"Error al crear rango de fechas para índice {indice} ({symbol}): {err}.")
                continue

            nombre_real = 'Ajuste Real'
            mostrar_leyenda_real = nombre_real not in leyendas_mostradas
            fig.add_trace(go.Scatter(x=puntos_tiempo, y=precios_reales, mode='lines', name=nombre_real,
                                     line=dict(color='mediumpurple', width=1.5),
                                     legendgroup='fit', showlegend=mostrar_leyenda_real), row=1, col=1)
            if mostrar_leyenda_real: leyendas_mostradas.add(nombre_real)

            if isinstance(curva_ideal, (list, np.ndarray)) and len(curva_ideal) == duracion_puntos:
                nombre_ideal = 'Ajuste Ideal'
                mostrar_leyenda_ideal = nombre_ideal not in leyendas_mostradas
                fig.add_trace(go.Scatter(x=puntos_tiempo, y=curva_ideal, mode='lines', name=nombre_ideal,
                                         line=dict(color='deepskyblue', width=1.5, dash='dot'),
                                         legendgroup='fit', showlegend=mostrar_leyenda_ideal), row=1, col=1)
                if mostrar_leyenda_ideal: leyendas_mostradas.add(nombre_ideal)

    columna_cambio_futuro = 'future_max_increase_capped'
    textos_hover = []
    for indice, fila in valles_confirmados.iterrows():
        ts_str = f"{fila['timestamp']:%Y-%m-%d %H:%M}" if pd.notna(fila['timestamp']) else "N/A"
        precio_str = f"{fila[columna_precio_base]:.4f}" if pd.notna(fila[columna_precio_base]) else "N/A"
        duracion_str = f"{fila.get('duration_minutes', 'N/A')}"
        mse_str = f"{fila.get('mse_fit', 0.0):.6f}"
        cambio_futuro_str = f"{fila.get(columna_cambio_futuro, 0.0)*100:.2f}%"

        textos_hover.append(
            f"<b>Target=1</b><br>" +
            f"Timestamp: {ts_str}<br>" +
            f"Precio: {precio_str}<br>" +
            f"Duración: {duracion_str} min<br>" +
            f"MSE Fit: {mse_str}<br>" +
            f"Subida Futura Máx: {cambio_futuro_str}" +
            f"<extra></extra>"
        )

    fig.add_trace(go.Scatter(
        x=valles_confirmados['timestamp'], y=valles_confirmados[columna_precio_base],
        mode='markers', name='Target = 1',
        marker=dict(size=8, color='blue', symbol='star', line=dict(width=1, color='black')),
        hovertemplate=textos_hover, legendgroup='markers'
    ), row=1, col=1)

    fig.add_trace(go.Bar(x=df_simbolo['timestamp'], y=df_simbolo['volume'],
                         marker=dict(color='rgba(150, 150, 150, 0.5)'), name='Volumen'), row=2, col=1)

    fig.update_layout(
        title_text=f"Valles Confirmados (Target=1) - {symbol} {('('+date_str+')') if date_str else ''}",
        xaxis_title="Timestamp",
        yaxis_title="Precio",
        yaxis2_title="Volumen",
        height=500, # Modificado
        width=1000, # Modificado
        legend_title_text="Leyenda",
        xaxis_rangeslider_visible=False,
        hovermode='x unified'
    )
    fig.show()

    try:
        directorio_salida = 'data/images_target_detailed'
        os.makedirs(directorio_salida, exist_ok=True)
        nombre_archivo = f"{symbol}_target_detailed_{date_str}.png" if date_str else f"{symbol}_target_detailed.png"
        ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
        fig.write_image(ruta_imagen, scale=1) # Modificado
        logging.info(f"Gráfico guardado: {ruta_imagen}")
    except Exception as err:
        logging.error(f"Error al guardar gráfico detallado para {symbol}: {err}")

logging.info("Función 'plot_confirmed_valleys_detailed' definida.")

2025-04-06 13:10:35,250 - INFO - Función 'plot_confirmed_valleys_detailed' definida.


In [11]:
def plot_all_sinusoid_shapes(df_processed, symbol, date_str=""):
    """
    Genera y guarda un gráfico de todas las formas sinusoidales detectadas para un símbolo.

    Args:
        df_processed (pd.DataFrame): DataFrame con los resultados del análisis.
        symbol (str): Símbolo a graficar.
        date_str (str): Etiqueta de fecha opcional para el título y nombre de archivo.

    Returns:
        None: La función muestra y guarda la gráfica.
    """
    logging.info(f"Generando gráfico de todas las formas sinusoidales para: {symbol}")

    df_simbolo = df_processed[df_processed['symbol'] == symbol].copy()
    if df_simbolo.empty:
        logging.warning(f"No hay datos para '{symbol}'.")
        return

    if not pd.api.types.is_datetime64_any_dtype(df_simbolo['timestamp']):
        df_simbolo['timestamp'] = pd.to_datetime(df_simbolo['timestamp'], errors='coerce')
        df_simbolo.dropna(subset=['timestamp'], inplace=True)
        if df_simbolo.empty:
            logging.warning(f"No quedan datos válidos para '{symbol}' tras conversión.")
            return

    puntos_sinusoidales = df_simbolo[df_simbolo['is_sinusoid_shape'] == 1].copy()
    if puntos_sinusoidales.empty:
        logging.info(f"No se encontraron formas sinusoidales para '{symbol}'.")
        return

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03,
                        row_heights=[0.8, 0.2],
                        subplot_titles=(f"{symbol} - Todas las Formas Detectadas", "Volumen"))

    columna_precio_base = 'close_lag_0'

    fig.add_trace(go.Scatter(x=df_simbolo['timestamp'], y=df_simbolo[columna_precio_base],
                             mode='lines', line=dict(color='darkgrey', width=1),
                             name='Precio', legendgroup='price'), row=1, col=1)

    leyendas_mostradas = set()
    logging.info(f"Graficando {len(puntos_sinusoidales)} formas sinusoidales para {symbol}...")

    for indice, fila in puntos_sinusoidales.iterrows():
        precios_reales = fila.get('actual_prices_fit', [])
        curva_ideal = fila.get('ideal_curve_fit', [])
        ts_inicio = fila.get('preceding_peak_ts')
        ts_fin = fila['timestamp']

        if isinstance(precios_reales, (list, np.ndarray)) and len(precios_reales) > 1 and pd.notna(ts_inicio) and pd.notna(ts_fin):
            duracion_puntos = len(precios_reales)
            try:
                puntos_tiempo = pd.date_range(ts_inicio, ts_fin, periods=duracion_puntos, name=f'fit_{indice}')
            except ValueError as err:
                logging.warning(f"Error al crear rango de fechas para índice {indice} ({symbol}): {err}.")
                continue

            nombre_real = 'Ajuste Real'
            mostrar_leyenda_real = nombre_real not in leyendas_mostradas
            fig.add_trace(go.Scatter(x=puntos_tiempo, y=precios_reales, mode='lines', name=nombre_real,
                                     line=dict(color='mediumpurple', width=1.5),
                                     legendgroup='fit', showlegend=mostrar_leyenda_real), row=1, col=1)
            if mostrar_leyenda_real: leyendas_mostradas.add(nombre_real)

            if isinstance(curva_ideal, (list, np.ndarray)) and len(curva_ideal) == duracion_puntos:
                nombre_ideal = 'Ajuste Ideal'
                mostrar_leyenda_ideal = nombre_ideal not in leyendas_mostradas
                fig.add_trace(go.Scatter(x=puntos_tiempo, y=curva_ideal, mode='lines', name=nombre_ideal,
                                         line=dict(color='deepskyblue', width=1.5, dash='dot'),
                                         legendgroup='fit', showlegend=mostrar_leyenda_ideal), row=1, col=1)
                if mostrar_leyenda_ideal: leyendas_mostradas.add(nombre_ideal)

    columna_cambio_futuro = 'future_max_increase_capped'
    textos_hover = []
    for indice, fila in puntos_sinusoidales.iterrows():
        ts_str = f"{fila['timestamp']:%Y-%m-%d %H:%M}" if pd.notna(fila['timestamp']) else "N/A"
        precio_str = f"{fila[columna_precio_base]:.4f}" if pd.notna(fila[columna_precio_base]) else "N/A"
        duracion_str = f"{fila.get('duration_minutes', 0)}"
        mse_str = f"{fila.get('mse_fit', 0.0):.6f}"
        target_val_str = f"{int(fila.get('target', 0))}"
        calc_fut_inc_str = f"{fila.get('future_increase_perc_calc', 0.0)*100:.2f}%"
        cambio_futuro_cap_str = f"{fila.get(columna_cambio_futuro, 0.0)*100:.2f}%"

        textos_hover.append(
            f"<b>Forma Detectada</b><br>" +
            f"Timestamp: {ts_str}<br>" +
            f"Precio: {precio_str}<br>" +
            f"Duración: {duracion_str} min<br>" +
            f"MSE Fit: {mse_str}<br>" +
            f"Target OK: {target_val_str}<br>" +
            f"Subida Calc: {calc_fut_inc_str}<br>" +
            f"Subida Máx: {cambio_futuro_cap_str}" +
            f"<extra></extra>"
        )

    fig.add_trace(go.Scatter(
        x=puntos_sinusoidales['timestamp'], y=puntos_sinusoidales[columna_precio_base],
        mode='markers', name='Forma Detectada',
        marker=dict(size=7, color='orange', symbol='circle', line=dict(width=1, color='black')),
        hovertemplate=textos_hover, legendgroup='markers'
    ), row=1, col=1)

    fig.add_trace(go.Bar(x=df_simbolo['timestamp'], y=df_simbolo['volume'],
                         marker=dict(color='rgba(150, 150, 150, 0.5)'), name='Volumen'), row=2, col=1)

    fig.update_layout(
        title_text=f"Todas las Formas Detectadas (Shape=1) - {symbol} {('('+date_str+')') if date_str else ''}",
        xaxis_title="Timestamp",
        yaxis_title="Precio",
        yaxis2_title="Volumen",
        height=500, # Modificado
        width=1000, # Modificado
        legend_title_text="Leyenda",
        xaxis_rangeslider_visible=False,
        hovermode='x unified'
    )
    fig.show()

    try:
        directorio_salida = 'data/images_sinusoid_shapes'
        os.makedirs(directorio_salida, exist_ok=True)
        nombre_archivo = f"{symbol}_all_shapes_{date_str}.png" if date_str else f"{symbol}_all_shapes.png"
        ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
        fig.write_image(ruta_imagen, scale=1) # Modificado
        logging.info(f"Gráfico guardado: {ruta_imagen}")
    except Exception as err:
        logging.error(f"Error al guardar gráfico de formas para {symbol}: {err}")

logging.info("Función 'plot_all_sinusoid_shapes' definida.")

2025-04-06 13:10:35,350 - INFO - Función 'plot_all_sinusoid_shapes' definida.


In [12]:
simbolos_disponibles = df_procesado['symbol'].unique()

if len(simbolos_disponibles) == 0:
    logging.warning("No hay símbolos disponibles en los datos procesados.")
else:
    logging.info(f"Símbolos disponibles: {len(simbolos_disponibles)}")
    logging.info("Calculando recuento de target=1 por símbolo...")

    recuentos_target = df_procesado[df_procesado['target'] == 1]['symbol'].value_counts()

    if recuentos_target.empty:
        logging.warning("No se encontraron instancias con target=1.")
    else:
        print("\nRecuento Target=1 por Símbolo (Top 20):")
        print(recuentos_target.head(20))

        num_top_simbolos = 5
        simbolos_a_graficar = recuentos_target.head(num_top_simbolos).index.tolist()

        if not simbolos_a_graficar:
            logging.warning("No hay símbolos con target=1 suficientes para graficar.")
        else:
            try:
                # Usar la primera fecha del df procesado como etiqueta general
                primer_ts_valido = df_procesado['timestamp'].dropna().iloc[0]
                etiqueta_fecha = pd.to_datetime(primer_ts_valido).strftime('%Y%m%d')
            except Exception:
                etiqueta_fecha = "date_unknown"
                logging.warning("No se pudo determinar etiqueta de fecha, usando 'date_unknown'.")

            logging.info(f"Generando gráficos detallados (target=1) para {len(simbolos_a_graficar)} símbolos principales: {', '.join(simbolos_a_graficar)}")

            conteo_graficos_generados = 0
            for simbolo_actual in simbolos_a_graficar:
                print("-" * 30)
                logging.info(f"Procesando gráfico para: {simbolo_actual}")
                try:
                    plot_confirmed_valleys_detailed(
                        df_processed=df_procesado,
                        symbol=simbolo_actual,
                        date_str=etiqueta_fecha
                    )
                    conteo_graficos_generados += 1
                except Exception as err_plot:
                    logging.error(f"Fallo al generar gráfico para {simbolo_actual}: {err_plot}")

            logging.info(f"Se generaron {conteo_graficos_generados} gráficos detallados de target=1.")

2025-04-06 13:10:35,476 - INFO - Símbolos disponibles: 310
2025-04-06 13:10:35,476 - INFO - Calculando recuento de target=1 por símbolo...
2025-04-06 13:10:35,515 - INFO - Generando gráficos detallados (target=1) para 5 símbolos principales: STPT, BEL, DODO, ENA, ARPA
2025-04-06 13:10:35,517 - INFO - Procesando gráfico para: STPT
2025-04-06 13:10:35,520 - INFO - Generando gráfico detallado (target=1) para: STPT



Recuento Target=1 por Símbolo (Top 20):
symbol
STPT      9
BEL       8
DODO      7
ENA       7
ARPA      6
XNO       5
SLF       5
HARD      4
TAO       4
ASR       4
BETA      4
KDA       3
PENDLE    3
QI        3
FET       3
CTK       3
WING      3
GTC       3
DYM       3
LAZIO     3
Name: count, dtype: int64
------------------------------


2025-04-06 13:10:36,443 - INFO - Graficando 9 valles confirmados para STPT...


2025-04-06 13:10:41,197 - INFO - Gráfico guardado: data/images_target_detailed\STPT_target_detailed_20250206.png
2025-04-06 13:10:41,202 - INFO - Procesando gráfico para: BEL
2025-04-06 13:10:41,202 - INFO - Generando gráfico detallado (target=1) para: BEL
2025-04-06 13:10:41,316 - INFO - Graficando 8 valles confirmados para BEL...


------------------------------


...

2025-04-06 13:10:42,824 - INFO - Gráfico guardado: data/images_target_detailed\DODO_target_detailed_20250206.png
2025-04-06 13:10:42,827 - INFO - Procesando gráfico para: ENA
2025-04-06 13:10:42,828 - INFO - Generando gráfico detallado (target=1) para: ENA
2025-04-06 13:10:42,939 - INFO - Graficando 7 valles confirmados para ENA...


------------------------------


2025-04-06 13:10:43,687 - INFO - Gráfico guardado: data/images_target_detailed\ENA_target_detailed_20250206.png
2025-04-06 13:10:43,689 - INFO - Procesando gráfico para: ARPA
2025-04-06 13:10:43,691 - INFO - Generando gráfico detallado (target=1) para: ARPA
2025-04-06 13:10:43,775 - INFO - Graficando 6 valles confirmados para ARPA...


------------------------------


2025-04-06 13:10:44,491 - INFO - Gráfico guardado: data/images_target_detailed\ARPA_target_detailed_20250206.png
2025-04-06 13:10:44,491 - INFO - Se generaron 5 gráficos detallados de target=1.


In [13]:
simbolo_especifico = "STPT" # Ejemplo, puede cambiarse

simbolos_disponibles = df_procesado['symbol'].unique()

if simbolo_especifico not in simbolos_disponibles:
    logging.warning(f"Símbolo '{simbolo_especifico}' no encontrado.")
    if len(simbolos_disponibles) > 0:
        logging.info(f"Primeros 20 símbolos disponibles: {list(simbolos_disponibles[:20])}")
    else:
        logging.info("No hay símbolos disponibles en el DataFrame.")
else:
    logging.info(f"Generando gráfico de todas las formas para: {simbolo_especifico}")

    try:
        df_simbolo_especifico = df_procesado[df_procesado['symbol'] == simbolo_especifico]
        primer_timestamp_valido = df_simbolo_especifico['timestamp'].dropna().iloc[0]
        etiqueta_fecha = pd.to_datetime(primer_timestamp_valido).strftime('%Y%m%d')
    except Exception:
        etiqueta_fecha = "date_unknown"
        logging.warning(f"No se pudo obtener fecha para {simbolo_especifico}. Usando etiqueta genérica.")

    print("-" * 30)
    try:
        plot_all_sinusoid_shapes(
            df_processed=df_procesado,
            symbol=simbolo_especifico,
            date_str=etiqueta_fecha
        )
        logging.info(f"Gráfico para '{simbolo_especifico}' mostrado/guardado.")

        if df_procesado[(df_procesado['symbol'] == simbolo_especifico) & (df_procesado['is_sinusoid_shape'] == 1)].empty:
            print(f"Nota: No se encontraron formas sinusoidales para '{simbolo_especifico}'.")

    except Exception as err_plot:
        logging.error(f"Fallo al generar gráfico de formas para {simbolo_especifico}: {err_plot}")

2025-04-06 13:10:44,609 - INFO - Generando gráfico de todas las formas para: STPT
2025-04-06 13:10:44,678 - INFO - Generando gráfico de todas las formas sinusoidales para: STPT
2025-04-06 13:10:44,808 - INFO - Graficando 46 formas sinusoidales para STPT...


------------------------------


2025-04-06 13:10:46,152 - INFO - Gráfico guardado: data/images_sinusoid_shapes\STPT_all_shapes_20250206.png
2025-04-06 13:10:46,152 - INFO - Gráfico para 'STPT' mostrado/guardado.


In [14]:
df_procesado.sample(5)

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_lag_0,volume_lag_0,open_lag_0,high_lag_0,low_lag_0,quote_asset_volume_lag_0,number_of_trades_lag_0,taker_buy_base_asset_volume_lag_0,taker_buy_quote_asset_volume_lag_0,close_lag_1,volume_lag_1,open_lag_1,high_lag_1,low_lag_1,quote_asset_volume_lag_1,number_of_trades_lag_1,taker_buy_base_asset_volume_lag_1,taker_buy_quote_asset_volume_lag_1,close_lag_2,volume_lag_2,open_lag_2,high_lag_2,low_lag_2,quote_asset_volume_lag_2,number_of_trades_lag_2,taker_buy_base_asset_volume_lag_2,taker_buy_quote_asset_volume_lag_2,close_lag_3,volume_lag_3,open_lag_3,high_lag_3,low_lag_3,quote_asset_volume_lag_3,number_of_trades_lag_3,taker_buy_base_asset_volume_lag_3,taker_buy_quote_asset_volume_lag_3,close_lag_4,volume_lag_4,open_lag_4,high_lag_4,low_lag_4,quote_asset_volume_lag_4,number_of_trades_lag_4,taker_buy_base_asset_volume_lag_4,taker_buy_quote_asset_volume_lag_4,close_lag_5,volume_lag_5,open_lag_5,high_lag_5,low_lag_5,quote_asset_volume_lag_5,number_of_trades_lag_5,taker_buy_base_asset_volume_lag_5,taker_buy_quote_asset_volume_lag_5,close_lag_6,volume_lag_6,open_lag_6,high_lag_6,low_lag_6,quote_asset_volume_lag_6,number_of_trades_lag_6,taker_buy_base_asset_volume_lag_6,taker_buy_quote_asset_volume_lag_6,close_lag_7,volume_lag_7,open_lag_7,high_lag_7,low_lag_7,quote_asset_volume_lag_7,number_of_trades_lag_7,taker_buy_base_asset_volume_lag_7,taker_buy_quote_asset_volume_lag_7,close_lag_8,volume_lag_8,open_lag_8,high_lag_8,low_lag_8,quote_asset_volume_lag_8,number_of_trades_lag_8,taker_buy_base_asset_volume_lag_8,taker_buy_quote_asset_volume_lag_8,close_lag_9,volume_lag_9,open_lag_9,high_lag_9,low_lag_9,quote_asset_volume_lag_9,number_of_trades_lag_9,taker_buy_base_asset_volume_lag_9,taker_buy_quote_asset_volume_lag_9,close_lag_10,volume_lag_10,open_lag_10,high_lag_10,low_lag_10,quote_asset_volume_lag_10,number_of_trades_lag_10,taker_buy_base_asset_volume_lag_10,taker_buy_quote_asset_volume_lag_10,close_lag_11,volume_lag_11,open_lag_11,high_lag_11,low_lag_11,quote_asset_volume_lag_11,number_of_trades_lag_11,taker_buy_base_asset_volume_lag_11,taker_buy_quote_asset_volume_lag_11,close_lag_12,volume_lag_12,open_lag_12,high_lag_12,low_lag_12,quote_asset_volume_lag_12,number_of_trades_lag_12,taker_buy_base_asset_volume_lag_12,taker_buy_quote_asset_volume_lag_12,close_lag_13,volume_lag_13,open_lag_13,high_lag_13,low_lag_13,quote_asset_volume_lag_13,number_of_trades_lag_13,taker_buy_base_asset_volume_lag_13,taker_buy_quote_asset_volume_lag_13,close_lag_14,volume_lag_14,open_lag_14,high_lag_14,low_lag_14,quote_asset_volume_lag_14,number_of_trades_lag_14,taker_buy_base_asset_volume_lag_14,taker_buy_quote_asset_volume_lag_14,close_lag_15,volume_lag_15,open_lag_15,high_lag_15,low_lag_15,quote_asset_volume_lag_15,number_of_trades_lag_15,taker_buy_base_asset_volume_lag_15,taker_buy_quote_asset_volume_lag_15,close_lag_16,volume_lag_16,open_lag_16,high_lag_16,low_lag_16,quote_asset_volume_lag_16,number_of_trades_lag_16,taker_buy_base_asset_volume_lag_16,taker_buy_quote_asset_volume_lag_16,close_lag_17,volume_lag_17,open_lag_17,high_lag_17,low_lag_17,quote_asset_volume_lag_17,number_of_trades_lag_17,taker_buy_base_asset_volume_lag_17,taker_buy_quote_asset_volume_lag_17,close_lag_18,volume_lag_18,open_lag_18,high_lag_18,low_lag_18,quote_asset_volume_lag_18,number_of_trades_lag_18,taker_buy_base_asset_volume_lag_18,taker_buy_quote_asset_volume_lag_18,close_lag_19,volume_lag_19,open_lag_19,high_lag_19,low_lag_19,quote_asset_volume_lag_19,number_of_trades_lag_19,taker_buy_base_asset_volume_lag_19,taker_buy_quote_asset_volume_lag_19,close_lag_20,volume_lag_20,open_lag_20,high_lag_20,low_lag_20,quote_asset_volume_lag_20,number_of_trades_lag_20,taker_buy_base_asset_volume_lag_20,taker_buy_quote_asset_volume_lag_20,close_lag_21,volume_lag_21,open_lag_21,high_lag_21,low_lag_21,quote_asset_volume_lag_21,number_of_trades_lag_

In [15]:
logging.info("Preparando DataFrame final para exportar...")

columnas_a_eliminar = [
    'valley_price_at_0',
    'duration_minutes',
    'mse_fit',
    'future_increase_perc_calc',
    'preceding_peak_lag',
    'preceding_peak_price',
    'actual_prices_fit',
    'ideal_curve_fit',
    'preceding_peak_ts'
]

df_final = df_procesado.drop(columns=columnas_a_eliminar, errors='ignore')

logging.info(f"Columnas intermedias eliminadas. Shape final: {df_final.shape}")

print("\nResumen de columnas finales:")
columnas_finales = df_final.columns.tolist()
if len(columnas_finales) > 30:
    print(f"Primeras 15: {columnas_finales[:15]}")
    print(f"Últimas 15: {columnas_finales[-15:]}")
    print(f"(Total: {len(columnas_finales)})")
else:
    print(columnas_finales)

print("\nPrimeras 5 filas del DataFrame final:")
print(df_final.head())

try:
    primer_ts_valido = df_final['timestamp'].dropna().iloc[0]
    etiqueta_fecha_exportacion = pd.to_datetime(primer_ts_valido).strftime('%Y%m%d')
except Exception:
    etiqueta_fecha_exportacion = "date_unknown"
    logging.warning("No se pudo determinar etiqueta de fecha para exportación.")

nombre_archivo_salida = f"df_final_target_{etiqueta_fecha_exportacion}.csv"

try:
    uso_memoria_mb = df_final.memory_usage(deep=True).sum() / (1024**2)
    logging.info(f"Tamaño estimado en memoria: {uso_memoria_mb:.2f} MB")
    logging.info(f"Guardando DataFrame final como: {nombre_archivo_salida}")

    df_final.to_csv(nombre_archivo_salida, index=False, chunksize=100000)

    logging.info(f"DataFrame final guardado exitosamente.")

except Exception as err_guardado:
    logging.error(f"Error al guardar el DataFrame final: {err_guardado}")

2025-04-06 13:10:47,338 - INFO - Preparando DataFrame final para exportar...
2025-04-06 13:10:50,641 - INFO - Columnas intermedias eliminadas. Shape final: (432450, 489)



Resumen de columnas finales:
Primeras 15: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'symbol', 'close_lag_0', 'volume_lag_0', 'open_lag_0', 'high_lag_0']
Últimas 15: ['PriceAccel_Momentum_Diff_10_20', 'PriceAccel_Momentum_Diff_10_25', 'PriceAccel_Momentum_Diff_10_30', 'PriceAccel_Momentum_Diff_15_20', 'PriceAccel_Momentum_Diff_15_25', 'PriceAccel_Momentum_Diff_15_30', 'PriceAccel_Momentum_Diff_20_25', 'PriceAccel_Momentum_Diff_20_30', 'PriceAccel_Momentum_Diff_25_30', 'VPT', 'MFI_14', 'MFI_20', 'future_max_increase_capped', 'target', 'is_sinusoid_shape']
(Total: 489)

Primeras 5 filas del DataFrame final:
            timestamp           open           high            low  \
0 2025-02-06 23:30:00 0.252500000000 0.252700000000 0.252500000000   
1 2025-02-06 23:31:00 0.252600000000 0.252900000000 0.252600000000   
2 2025-02-06 23:32:00 0.252900000000 0.252900000000 0.2526

2025-04-06 13:10:51,180 - INFO - Tamaño estimado en memoria: 1597.52 MB
2025-04-06 13:10:51,182 - INFO - Guardando DataFrame final como: df_final_target_20250206.csv
2025-04-06 13:16:01,597 - INFO - DataFrame final guardado exitosamente.


In [16]:
df_final.sample(5)

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_lag_0,volume_lag_0,open_lag_0,high_lag_0,low_lag_0,quote_asset_volume_lag_0,number_of_trades_lag_0,taker_buy_base_asset_volume_lag_0,taker_buy_quote_asset_volume_lag_0,close_lag_1,volume_lag_1,open_lag_1,high_lag_1,low_lag_1,quote_asset_volume_lag_1,number_of_trades_lag_1,taker_buy_base_asset_volume_lag_1,taker_buy_quote_asset_volume_lag_1,close_lag_2,volume_lag_2,open_lag_2,high_lag_2,low_lag_2,quote_asset_volume_lag_2,number_of_trades_lag_2,taker_buy_base_asset_volume_lag_2,taker_buy_quote_asset_volume_lag_2,close_lag_3,volume_lag_3,open_lag_3,high_lag_3,low_lag_3,quote_asset_volume_lag_3,number_of_trades_lag_3,taker_buy_base_asset_volume_lag_3,taker_buy_quote_asset_volume_lag_3,close_lag_4,volume_lag_4,open_lag_4,high_lag_4,low_lag_4,quote_asset_volume_lag_4,number_of_trades_lag_4,taker_buy_base_asset_volume_lag_4,taker_buy_quote_asset_volume_lag_4,close_lag_5,volume_lag_5,open_lag_5,high_lag_5,low_lag_5,quote_asset_volume_lag_5,number_of_trades_lag_5,taker_buy_base_asset_volume_lag_5,taker_buy_quote_asset_volume_lag_5,close_lag_6,volume_lag_6,open_lag_6,high_lag_6,low_lag_6,quote_asset_volume_lag_6,number_of_trades_lag_6,taker_buy_base_asset_volume_lag_6,taker_buy_quote_asset_volume_lag_6,close_lag_7,volume_lag_7,open_lag_7,high_lag_7,low_lag_7,quote_asset_volume_lag_7,number_of_trades_lag_7,taker_buy_base_asset_volume_lag_7,taker_buy_quote_asset_volume_lag_7,close_lag_8,volume_lag_8,open_lag_8,high_lag_8,low_lag_8,quote_asset_volume_lag_8,number_of_trades_lag_8,taker_buy_base_asset_volume_lag_8,taker_buy_quote_asset_volume_lag_8,close_lag_9,volume_lag_9,open_lag_9,high_lag_9,low_lag_9,quote_asset_volume_lag_9,number_of_trades_lag_9,taker_buy_base_asset_volume_lag_9,taker_buy_quote_asset_volume_lag_9,close_lag_10,volume_lag_10,open_lag_10,high_lag_10,low_lag_10,quote_asset_volume_lag_10,number_of_trades_lag_10,taker_buy_base_asset_volume_lag_10,taker_buy_quote_asset_volume_lag_10,close_lag_11,volume_lag_11,open_lag_11,high_lag_11,low_lag_11,quote_asset_volume_lag_11,number_of_trades_lag_11,taker_buy_base_asset_volume_lag_11,taker_buy_quote_asset_volume_lag_11,close_lag_12,volume_lag_12,open_lag_12,high_lag_12,low_lag_12,quote_asset_volume_lag_12,number_of_trades_lag_12,taker_buy_base_asset_volume_lag_12,taker_buy_quote_asset_volume_lag_12,close_lag_13,volume_lag_13,open_lag_13,high_lag_13,low_lag_13,quote_asset_volume_lag_13,number_of_trades_lag_13,taker_buy_base_asset_volume_lag_13,taker_buy_quote_asset_volume_lag_13,close_lag_14,volume_lag_14,open_lag_14,high_lag_14,low_lag_14,quote_asset_volume_lag_14,number_of_trades_lag_14,taker_buy_base_asset_volume_lag_14,taker_buy_quote_asset_volume_lag_14,close_lag_15,volume_lag_15,open_lag_15,high_lag_15,low_lag_15,quote_asset_volume_lag_15,number_of_trades_lag_15,taker_buy_base_asset_volume_lag_15,taker_buy_quote_asset_volume_lag_15,close_lag_16,volume_lag_16,open_lag_16,high_lag_16,low_lag_16,quote_asset_volume_lag_16,number_of_trades_lag_16,taker_buy_base_asset_volume_lag_16,taker_buy_quote_asset_volume_lag_16,close_lag_17,volume_lag_17,open_lag_17,high_lag_17,low_lag_17,quote_asset_volume_lag_17,number_of_trades_lag_17,taker_buy_base_asset_volume_lag_17,taker_buy_quote_asset_volume_lag_17,close_lag_18,volume_lag_18,open_lag_18,high_lag_18,low_lag_18,quote_asset_volume_lag_18,number_of_trades_lag_18,taker_buy_base_asset_volume_lag_18,taker_buy_quote_asset_volume_lag_18,close_lag_19,volume_lag_19,open_lag_19,high_lag_19,low_lag_19,quote_asset_volume_lag_19,number_of_trades_lag_19,taker_buy_base_asset_volume_lag_19,taker_buy_quote_asset_volume_lag_19,close_lag_20,volume_lag_20,open_lag_20,high_lag_20,low_lag_20,quote_asset_volume_lag_20,number_of_trades_lag_20,taker_buy_base_asset_volume_lag_20,taker_buy_quote_asset_volume_lag_20,close_lag_21,volume_lag_21,open_lag_21,high_lag_21,low_lag_21,quote_asset_volume_lag_21,number_of_trades_lag_

In [17]:
df_final.shape

(432450, 489)

In [18]:
logging.info("Calculando tasa de éxito por patrón...")

columnas_patrones_velas_buscar = [
    'Doji', 'Hammer', 'HangingMan', 'BullishEngulfing', 'BearishEngulfing',
    'MorningStar', 'EveningStar', 'PiercingLine', 'DarkCloudCover',
    'ThreeWhiteSoldiers', 'ThreeBlackCrows'
]

columnas_velas_existentes = [
    col for col in columnas_patrones_velas_buscar if col in df_procesado.columns
]

patron_sinusoidal_valido = ('is_sinusoid_shape' in df_procesado.columns and
                            'target' in df_procesado.columns)

if not columnas_velas_existentes and not patron_sinusoidal_valido:
    logging.warning("No se encontraron columnas de patrones de velas ni datos para patrón sinusoidal.")
    df_tasas_exito = pd.DataFrame() # Crear DataFrame vacío si no hay nada que analizar
else:
    patrones_analizados = []
    if columnas_velas_existentes:
        patrones_analizados.extend(columnas_velas_existentes)
    if patron_sinusoidal_valido:
        patrones_analizados.append("Sinusoidal")
    logging.info(f"Analizando patrones: {', '.join(patrones_analizados)}")

    columna_incremento = 'future_max_increase_capped'
    if columna_incremento not in df_procesado.columns:
         logging.error(f"Columna '{columna_incremento}' no encontrada. No se puede calcular éxito.")
         df_tasas_exito = pd.DataFrame()
    else:
        condicion_exito_general = (df_procesado[columna_incremento] >= MIN_FUTURE_INCREASE_PERCENT)
        tasas_exito = {}

        for nombre_columna_patron in columnas_velas_existentes:
            # Asegurar que la columna es numérica (0 o 1)
            if df_procesado[nombre_columna_patron].dtype == 'bool':
                columna_patron_int = df_procesado[nombre_columna_patron].astype(int)
            elif not pd.api.types.is_numeric_dtype(df_procesado[nombre_columna_patron]):
                columna_patron_int = pd.to_numeric(df_procesado[nombre_columna_patron], errors='coerce').fillna(0).astype(int)
            else:
                columna_patron_int = df_procesado[nombre_columna_patron].fillna(0)

            mascara_ocurre_patron = (columna_patron_int == 1)
            total_ocurrencias_patron = mascara_ocurre_patron.sum()
            etiqueta_patron = nombre_columna_patron

            if total_ocurrencias_patron > 0:
                ocurrencias_exitosas = (mascara_ocurre_patron & condicion_exito_general).sum()
                tasa = (ocurrencias_exitosas / total_ocurrencias_patron) * 100
                tasas_exito[etiqueta_patron] = {
                    'rate': tasa,
                    'count': total_ocurrencias_patron,
                    'success_count': ocurrencias_exitosas
                }
            else:
                tasas_exito[etiqueta_patron] = {'rate': 0.0, 'count': 0, 'success_count': 0}

        if patron_sinusoidal_valido:
            etiqueta_sinusoidal = "Sinusoidal"
            mascara_forma_detectada = (df_procesado['is_sinusoid_shape'] == 1)
            total_formas_detectadas = mascara_forma_detectada.sum()
            # Éxito para sinusoidal es target=1 (que ya implica subida futura)
            total_targets_confirmados = df_procesado['target'].sum()

            if total_formas_detectadas > 0:
                # Tasa de éxito: de las formas detectadas, cuántas cumplieron el target de subida
                tasa_sinusoidal = (total_targets_confirmados / total_formas_detectadas) * 100
                tasas_exito[etiqueta_sinusoidal] = {
                    'rate': tasa_sinusoidal,
                    'count': total_formas_detectadas, # Ocurrencia total es is_sinusoid_shape=1
                    'success_count': total_targets_confirmados # Éxito es target=1
                }
            else:
                tasas_exito[etiqueta_sinusoidal] = {'rate': 0.0, 'count': 0, 'success_count': 0}

        df_tasas_exito = pd.DataFrame.from_dict(tasas_exito, orient='index')
        df_tasas_exito = df_tasas_exito.sort_values(by='rate', ascending=False)
        logging.info("Cálculo de tasas de éxito completado.")

2025-04-06 13:16:03,249 - INFO - Calculando tasa de éxito por patrón...
2025-04-06 13:16:03,255 - INFO - Analizando patrones: Doji, Hammer, HangingMan, BullishEngulfing, BearishEngulfing, MorningStar, EveningStar, PiercingLine, DarkCloudCover, ThreeWhiteSoldiers, ThreeBlackCrows, Sinusoidal
2025-04-06 13:16:03,378 - INFO - Cálculo de tasas de éxito completado.


In [19]:
if 'df_tasas_exito' in locals() and not df_tasas_exito.empty:
    df_tasas_exito.head() # Mostrar las mejores tasas
else:
    print("No hay datos de tasas de éxito para mostrar.")

In [20]:
logging.info("Generando gráfico: conteo de target=1 por símbolo...")

conteo_target_por_simbolo = df_procesado[df_procesado['target'] == 1]['symbol'].value_counts()

if conteo_target_por_simbolo.empty:
    logging.warning("No hay datos con target=1 para generar gráfico de conteo.")
else:
    simbolos_a_mostrar = 30
    conteo_a_graficar = conteo_target_por_simbolo.head(simbolos_a_mostrar)

    figura_barras = go.Figure()

    figura_barras.add_trace(go.Bar(
        x=conteo_a_graficar.index,
        y=conteo_a_graficar.values,
        name='Conteo Target=1',
        marker_color='royalblue',
        hovertemplate='Símbolo: %{x}<br>Conteo: %{y:,}<extra></extra>'
    ))

    promedio_conteo = conteo_a_graficar.mean()
    figura_barras.add_hline(
        y=promedio_conteo,
        line_dash="dot",
        line_color="grey",
        annotation_text=f"Promedio (Top {simbolos_a_mostrar}): {promedio_conteo:.1f}",
        annotation_position="bottom right"
    )

    titulo_grafico = f'Conteo Target=1 por Símbolo (Top {simbolos_a_mostrar})'
    figura_barras.update_layout(
        title=titulo_grafico,
        xaxis_title='Símbolo',
        yaxis_title='Número de Ocurrencias (Target=1)',
        xaxis={'categoryorder':'total descending'},
        height=500,
        width=1000,
        showlegend=False
    )
    figura_barras.show()

    try:
        directorio_salida = 'data/images_analysis'
        os.makedirs(directorio_salida, exist_ok=True)
        nombre_archivo = f"target1_count_by_symbol_top{simbolos_a_mostrar}.png"
        ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
        figura_barras.write_image(ruta_imagen, scale=1)
        logging.info(f"Gráfico guardado: {ruta_imagen}")
    except Exception as err:
        logging.error(f"Error al guardar gráfico de conteo: {err}")

2025-04-06 13:16:03,529 - INFO - Generando gráfico: conteo de target=1 por símbolo...


2025-04-06 13:16:08,907 - INFO - Gráfico guardado: data/images_analysis\target1_count_by_symbol_top30.png


In [32]:
logging.info("Generando histograma: distribución del incremento futuro para target=1...")

datos_target_1 = df_procesado[df_procesado['target'] == 1].copy()

if datos_target_1.empty:
    logging.warning("No hay datos con target=1 para generar histograma.")
else:
    columna_incremento_futuro = 'future_max_increase_capped'
    if columna_incremento_futuro not in datos_target_1.columns:
        logging.error(f"Columna '{columna_incremento_futuro}' no encontrada en datos target=1.")
    else:
        datos_target_1['incremento_futuro_porcentaje'] = datos_target_1[columna_incremento_futuro] * 100

        figura_histograma = go.Figure()

        figura_histograma.add_trace(go.Histogram(
            x=datos_target_1['incremento_futuro_porcentaje'],
            name='Distribución',
            marker_color='lightseagreen',
            xbins=dict(
                start=MIN_FUTURE_INCREASE_PERCENT * 100,
                size=0.10 # Agrupando un poco más
            ),
            hovertemplate='Incremento: %{x:.2f}%<br>Frecuencia: %{y:,}<extra></extra>'
        ))

        titulo_grafico = 'Distribución Incremento Futuro (%) para Target=1'
        figura_histograma.update_layout(
            title=titulo_grafico,
            xaxis_title=f'Incremento Futuro Máx ({START_LEAD_FOR_CHANGE}-{END_LEAD_FOR_CHANGE}m, Cap {CAP_LIMIT*100:.0f}%) [%]',
            yaxis_title='Frecuencia (Ocurrencias)',
            bargap=0.1,
            height=500,
            width=1000
        )
        figura_histograma.show()

        try:
            directorio_salida = 'data/images_analysis'
            os.makedirs(directorio_salida, exist_ok=True)
            nombre_archivo = "target1_future_increase_histogram.png"
            ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
            figura_histograma.write_image(ruta_imagen, scale=1)
            logging.info(f"Gráfico guardado: {ruta_imagen}")
        except Exception as err:
            logging.error(f"Error al guardar histograma de incremento: {err}")

2025-04-06 13:22:38,105 - INFO - Generando histograma: distribución del incremento futuro para target=1...


2025-04-06 13:22:38,240 - INFO - Gráfico guardado: data/images_analysis\target1_future_increase_histogram.png


In [31]:
# %%
logging.info("Generando strip plot: incremento futuro por símbolo para target=1...")

datos_target_1 = df_procesado[df_procesado['target'] == 1].copy()

if datos_target_1.empty:
    logging.warning("No hay datos con target=1 para generar strip plot.")
else:
    conteo_target_simbolo = datos_target_1['symbol'].value_counts()
    # Asegurar que tomamos como máximo 30 símbolos, como en la imagen
    simbolos_a_graficar = conteo_target_simbolo.head(30).index.tolist()

    if not simbolos_a_graficar:
        logging.warning("No se encontraron símbolos con target=1 para graficar.")
    else:
        etiqueta_titulo = f"Top {len(simbolos_a_graficar)}"
        datos_grafico = datos_target_1[datos_target_1['symbol'].isin(simbolos_a_graficar)].copy()

        columna_incremento = 'future_max_increase_capped'
        if columna_incremento not in datos_grafico.columns:
            logging.error(f"Columna '{columna_incremento}' no encontrada para strip plot.")
        else:
            datos_grafico['incremento_porcentaje'] = datos_grafico[columna_incremento] * 100

            figura_strip = go.Figure()

            # --- Inicio Bloque Original Restaurado ---
            for simbolo in simbolos_a_graficar:
                datos_simbolo = datos_grafico[datos_grafico['symbol'] == simbolo]
                figura_strip.add_trace(go.Scatter(
                    x=[simbolo] * len(datos_simbolo),
                    y=datos_simbolo['incremento_porcentaje'],
                    mode='markers',
                    name=simbolo, # Aunque no se muestre la leyenda, es útil tenerlo
                    marker=dict(size=5, opacity=0.7),
                    # Añadir hovertemplate para info al pasar el ratón
                    hovertemplate=f"<b>Símbolo: {simbolo}</b><br>Incremento: %{{y:.2f}}%<extra></extra>"
                ))
            # --- Fin Bloque Original Restaurado ---

            titulo_grafico = f'Distribución Incremento Futuro (%) por Símbolo (Target=1, {etiqueta_titulo})'
            figura_strip.update_layout(
                title=titulo_grafico,
                yaxis_title=f'Incremento Futuro Máx ({START_LEAD_FOR_CHANGE}-{END_LEAD_FOR_CHANGE}m, Cap {CAP_LIMIT*100:.0f}%) [%]',
                xaxis_title='Símbolo',
                showlegend=False, # Mantener leyenda oculta como en el original
                # Asegurar orden correcto en eje X
                xaxis={'categoryorder':'array', 'categoryarray': simbolos_a_graficar},
                height=500, # Altura estándar
                width=1000, # Ancho estándar
                # Asegurar que las etiquetas del eje X estén rotadas si es necesario (automático usualmente)
                xaxis_tickangle=-90
            )
            figura_strip.show()

            try:
                directorio_salida = 'data/images_analysis'
                os.makedirs(directorio_salida, exist_ok=True)
                # Nombre de archivo consistente
                nombre_archivo = f"target1_future_increase_strip_by_symbol_top{len(simbolos_a_graficar)}.png"
                ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
                # Guardar con las dimensiones correctas
                figura_strip.write_image(ruta_imagen, width=1000, height=500, scale=1)
                logging.info(f"Gráfico guardado: {ruta_imagen}")
            except Exception as err:
                logging.error(f"Error al guardar strip plot: {err}")

2025-04-06 13:22:22,398 - INFO - Generando strip plot: incremento futuro por símbolo para target=1...


2025-04-06 13:22:22,995 - INFO - Gráfico guardado: data/images_analysis\target1_future_increase_strip_by_symbol_top30.png


In [23]:
logging.info("Generando gráfico: distribución horaria de target=1...")

datos_target_1 = df_procesado[df_procesado['target'] == 1].copy()

if datos_target_1.empty:
    logging.warning("No hay datos con target=1 para generar gráfico horario.")
else:
    if not pd.api.types.is_datetime64_any_dtype(datos_target_1['timestamp']):
        datos_target_1['timestamp'] = pd.to_datetime(datos_target_1['timestamp'], errors='coerce')
        datos_target_1.dropna(subset=['timestamp'], inplace=True)

    if datos_target_1.empty:
         logging.warning("No quedan datos válidos con target=1 tras verificar timestamp.")
    else:
        intervalo_minutos = 15 # Agrupar cada 15 minutos
        datos_target_1['etiqueta_intervalo_tiempo'] = (
            datos_target_1['timestamp'].dt.hour.apply(lambda h: f"{h:02d}") +
            ":" +
            (datos_target_1['timestamp'].dt.minute // intervalo_minutos * intervalo_minutos).apply(lambda m: f"{m:02d}")
        )

        conteo_intervalo_horario = datos_target_1['etiqueta_intervalo_tiempo'].value_counts().sort_index()

        figura_tiempo = go.Figure()

        figura_tiempo.add_trace(go.Bar(
            x=conteo_intervalo_horario.index,
            y=conteo_intervalo_horario.values,
            name='Conteo Target=1',
            marker_color='purple',
            hovertemplate='Intervalo: %{x}<br>Conteo: %{y:,}<extra></extra>'
        ))

        promedio_intervalo_conteo = conteo_intervalo_horario.mean()
        figura_tiempo.add_hline(
            y=promedio_intervalo_conteo,
            line_dash="dot",
            line_color="grey",
            annotation_text=f"Promedio={promedio_intervalo_conteo:.1f}",
            annotation_position="bottom right"
        )

        titulo_grafico = f'Distribución Target=1 por Intervalo de {intervalo_minutos} Minutos'
        figura_tiempo.update_layout(
            title=titulo_grafico,
            xaxis_title='Hora del Día (Intervalo)',
            yaxis_title='Número de Ocurrencias (Target=1)',
            xaxis=dict(type='category', tickangle=-45),
            height=500,
            width=1000,
            showlegend=False,
            bargap=0.2
        )
        figura_tiempo.show()

        try:
            directorio_salida = 'data/images_analysis'
            os.makedirs(directorio_salida, exist_ok=True)
            nombre_archivo = f"target1_hourly_distribution_{intervalo_minutos}min.png"
            ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
            figura_tiempo.write_image(ruta_imagen, scale=1)
            logging.info(f"Gráfico guardado: {ruta_imagen}")
        except Exception as err:
            logging.error(f"Error al guardar gráfico horario: {err}")

2025-04-06 13:16:09,689 - INFO - Generando gráfico: distribución horaria de target=1...


2025-04-06 13:16:09,877 - INFO - Gráfico guardado: data/images_analysis\target1_hourly_distribution_15min.png


In [24]:
logging.info("Generando gráfico Donut: proporción Target=1 vs Subida >= {MIN_FUTURE_INCREASE_PERCENT*100:.1f}%...")

conteo_target_1 = df_procesado['target'].sum()
columna_incremento = 'future_max_increase_capped'

if columna_incremento not in df_procesado.columns:
    logging.error(f"Columna '{columna_incremento}' no encontrada para gráfico Donut.")
else:
    mascara_subida_minima = (df_procesado[columna_incremento] >= MIN_FUTURE_INCREASE_PERCENT)
    conteo_subida_minima = mascara_subida_minima.sum()

    if conteo_subida_minima <= 0:
        logging.warning(f"No hay instancias con subida futura >= {MIN_FUTURE_INCREASE_PERCENT*100:.1f}%. No se genera gráfico Donut.")
    else:
        # Solo contar subidas sin target si target=1 implica subida (como en este caso)
        conteo_subida_sin_target = max(0, conteo_subida_minima - conteo_target_1)

        etiquetas = [f'Target=1', f'Subida ≥ {MIN_FUTURE_INCREASE_PERCENT*100:.1f}% (Sin Target=1)']
        valores = [conteo_target_1, conteo_subida_sin_target]
        colores = ['#1f77b4', '#ff7f0e'] # Azul, Naranja

        figura_donut = go.Figure()

        figura_donut.add_trace(go.Pie(
            labels=etiquetas,
            values=valores,
            hole=0.4,
            marker_colors=colores,
            pull=[0.05, 0], # Resaltar Target=1
            hoverinfo='label+percent+value',
            textinfo='percent+label', # Mostrar etiqueta y %
            textfont_size=12, # Reducir tamaño fuente si es necesario
            insidetextorientation='radial'
        ))

        porcentaje_target_de_subida = (conteo_target_1 / conteo_subida_minima) * 100

        titulo_grafico = f'Proporción Target=1 vs Subidas ≥ {MIN_FUTURE_INCREASE_PERCENT*100:.1f}%'
        figura_donut.update_layout(
            title_text=f'{titulo_grafico}<br>(Target=1 = {porcentaje_target_de_subida:.2f}% de Subidas ≥ {MIN_FUTURE_INCREASE_PERCENT*100:.1f}%)',
            legend_title_text="Categoría",
            annotations=[dict(text=f'Subidas≥{MIN_FUTURE_INCREASE_PERCENT*100:.1f}%<br>{conteo_subida_minima:,}',
                              x=0.5, y=0.5, font_size=16, showarrow=False)],
            height=500,
            width=1000,
            showlegend=True # Mostrar leyenda
        )
        figura_donut.show()

        try:
            directorio_salida = 'data/images_analysis'
            os.makedirs(directorio_salida, exist_ok=True)
            nombre_archivo = "target1_vs_min_increase_donut.png"
            ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
            figura_donut.write_image(ruta_imagen, scale=1)
            logging.info(f"Gráfico guardado: {ruta_imagen}")
        except Exception as err:
            logging.error(f"Error al guardar gráfico Donut: {err}")

2025-04-06 13:16:09,910 - INFO - Generando gráfico Donut: proporción Target=1 vs Subida >= {MIN_FUTURE_INCREASE_PERCENT*100:.1f}%...


2025-04-06 13:16:10,180 - INFO - Gráfico guardado: data/images_analysis\target1_vs_min_increase_donut.png


In [25]:
logging.info("Generando gráfico: comparación de conteos totales...")

total_filas = len(df_procesado)
columna_incremento = 'future_max_increase_capped'
total_subidas_minimas = 0
if columna_incremento in df_procesado.columns:
    total_subidas_minimas = (df_procesado[columna_incremento] >= MIN_FUTURE_INCREASE_PERCENT).sum()
else:
    logging.warning(f"Columna '{columna_incremento}' no encontrada para conteo de subidas.")

total_formas_detectadas = 0
if 'is_sinusoid_shape' in df_procesado.columns:
    total_formas_detectadas = df_procesado['is_sinusoid_shape'].sum()
else:
     logging.warning("Columna 'is_sinusoid_shape' no encontrada.")

total_targets_confirmados = 0
if 'target' in df_procesado.columns:
    total_targets_confirmados = df_procesado['target'].sum()
else:
     logging.warning("Columna 'target' no encontrada.")

etiquetas_conteos = ['Total Filas', f'Subida ≥ {MIN_FUTURE_INCREASE_PERCENT*100:.1f}%', 'Forma Detectada', 'Target=1']
valores_conteos = [total_filas, total_subidas_minimas, total_formas_detectadas, total_targets_confirmados]

colores_barras = ['lightslategray', 'lightcoral', 'lightskyblue', 'mediumseagreen']

figura_comparacion = go.Figure(go.Bar(
    x=etiquetas_conteos,
    y=valores_conteos,
    text=valores_conteos,
    textposition='auto',
    marker_color=colores_barras,
    hovertemplate='Evento: %{x}<br>Conteo: %{y:,}<extra></extra>'
))

figura_comparacion.update_traces(texttemplate='%{text:,}')

titulo_grafico = 'Comparación de Conteos Totales'
figura_comparacion.update_layout(
    title=titulo_grafico,
    yaxis_title='Número de Ocurrencias (Log)',
    xaxis_title='Tipo de Evento / Filtro',
    yaxis_type="log",
    height=500,
    width=1000
)
figura_comparacion.show()

try:
    directorio_salida = 'data/images_analysis'
    os.makedirs(directorio_salida, exist_ok=True)
    nombre_archivo = "total_event_counts_comparison.png"
    ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
    figura_comparacion.write_image(ruta_imagen, scale=1)
    logging.info(f"Gráfico guardado: {ruta_imagen}")
except Exception as err:
    logging.error(f"Error al guardar gráfico de comparación de conteos: {err}")

2025-04-06 13:16:10,223 - INFO - Generando gráfico: comparación de conteos totales...


2025-04-06 13:16:10,411 - INFO - Gráfico guardado: data/images_analysis\total_event_counts_comparison.png


In [34]:
# %%
logging.info("Generando gráfico horizontal (escala lineal): comparación de conteos totales...")

total_filas = len(df_procesado)
columna_incremento = 'future_max_increase_capped'
total_subidas_minimas = 0
if columna_incremento in df_procesado.columns:
    total_subidas_minimas = (df_procesado[columna_incremento] >= MIN_FUTURE_INCREASE_PERCENT).sum()
else:
    logging.warning(f"Columna '{columna_incremento}' no encontrada para conteo de subidas.")

total_formas_detectadas = 0
if 'is_sinusoid_shape' in df_procesado.columns:
    total_formas_detectadas = df_procesado['is_sinusoid_shape'].sum()
else:
     logging.warning("Columna 'is_sinusoid_shape' no encontrada.")

total_targets_confirmados = 0
if 'target' in df_procesado.columns:
    total_targets_confirmados = df_procesado['target'].sum()
else:
     logging.warning("Columna 'target' no encontrada.")

etiquetas_conteos = ['Total Filas', f'Subida ≥ {MIN_FUTURE_INCREASE_PERCENT*100:.1f}%', 'Forma Detectada', 'Target=1']
valores_conteos = [total_filas, total_subidas_minimas, total_formas_detectadas, total_targets_confirmados]

colores_barras = ['lightslategray', 'lightcoral', 'lightskyblue', 'mediumseagreen']

figura_comparacion = go.Figure(go.Bar(
    y=etiquetas_conteos, # Etiquetas en el eje Y
    x=valores_conteos,   # Valores en el eje X
    text=valores_conteos, # Texto a mostrar
    orientation='h',     # Orientación horizontal
    textposition='auto',
    marker_color=colores_barras,
    hovertemplate='Evento: %{y}<br>Conteo: %{x:,}<extra></extra>'
))

# Mostrar valores formateados
figura_comparacion.update_traces(texttemplate='%{x:,}')

titulo_grafico = 'Comparación de Conteos Totales (Escala Lineal)'
figura_comparacion.update_layout(
    title=titulo_grafico,
    xaxis_title='Número de Ocurrencias', # Título eje X (valores) - Escala Lineal
    yaxis_title='Tipo de Evento / Filtro', # Título eje Y (etiquetas)
    # xaxis_type="log", # <--- Línea eliminada para usar escala lineal
    yaxis=dict(autorange="reversed"), # Mostrar 'Total Filas' arriba
    height=500,
    width=1000
)
figura_comparacion.show()

try:
    directorio_salida = 'data/images_analysis'
    os.makedirs(directorio_salida, exist_ok=True)
    # Modificar nombre para reflejar el cambio a lineal
    nombre_archivo = "total_event_counts_comparison_linear_horizontal.png"
    ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
    figura_comparacion.write_image(ruta_imagen, width=1000, height=500, scale=1)
    logging.info(f"Gráfico guardado: {ruta_imagen}")
except Exception as err:
    logging.error(f"Error al guardar gráfico de comparación de conteos: {err}")

2025-04-06 13:25:06,550 - INFO - Generando gráfico horizontal (escala lineal): comparación de conteos totales...


2025-04-06 13:25:06,723 - INFO - Gráfico guardado: data/images_analysis\total_event_counts_comparison_linear_horizontal.png


In [26]:
logging.info("Generando gráfico: Ocurrencias vs Éxito por Patrón...")

if 'df_tasas_exito' not in locals() or not isinstance(df_tasas_exito, pd.DataFrame) or df_tasas_exito.empty:
    logging.warning("'df_tasas_exito' no disponible o vacío. No se genera gráfico comparativo.")
else:
    df_grafico = df_tasas_exito.sort_values(by='rate', ascending=False).copy()

    figura_comparativa_patrones = go.Figure()

    figura_comparativa_patrones.add_trace(go.Bar(
        name='Ocurrencias Totales',
        x=df_grafico.index,
        y=df_grafico['count'],
        text=df_grafico['count'].apply(lambda x: f'{x:,}' if x > 0 else ''),
        textposition='outside',
        marker_color='lightskyblue',
        offsetgroup=1,
        customdata=df_grafico['count'],
        hovertemplate="<b>%{x}</b><br>Total Ocurrencias: %{customdata:,}<extra></extra>"
    ))

    customdata_exito = df_grafico[['success_count', 'rate']].copy()
    customdata_exito['rate_str'] = customdata_exito['rate'].apply(lambda r: f"{r:.2f}%")

    figura_comparativa_patrones.add_trace(go.Bar(
        name='Ocurrencias con Éxito',
        x=df_grafico.index,
        y=df_grafico['success_count'],
        text=df_grafico['success_count'].apply(lambda x: f'{x:,}' if x > 0 else ''),
        textposition='auto',
        marker_color='mediumseagreen',
        offsetgroup=1,
        base=0,
        customdata=customdata_exito[['success_count', 'rate_str']],
        hovertemplate=("<b>%{x}</b><br>" +
                       "Éxito: %{customdata[0]:,}<br>" +
                       "Tasa Éxito: %{customdata[1]}" +
                       "<extra></extra>")
    ))

    titulo_grafico = 'Comparativa: Ocurrencias Totales vs. Éxito por Patrón'
    figura_comparativa_patrones.update_layout(
        title=titulo_grafico,
        xaxis_title='Patrón',
        yaxis_title='Número de Ocurrencias (Log)',
        barmode='group',
        yaxis_type="log",
        xaxis={'categoryorder':'array', 'categoryarray': df_grafico.index},
        legend_title_text="Tipo",
        height=500, # Ajustado
        width=1000,
        bargap=0.15,
        bargroupgap=0.1
    )
    figura_comparativa_patrones.show()

    try:
        directorio_salida = 'data/images_analysis'
        os.makedirs(directorio_salida, exist_ok=True)
        nombre_archivo = "pattern_counts_vs_success_comparison.png"
        ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
        figura_comparativa_patrones.write_image(ruta_imagen, scale=1)
        logging.info(f"Gráfico guardado: {ruta_imagen}")
    except Exception as err:
        logging.error(f"Error al guardar gráfico comparativo de patrones: {err}")

2025-04-06 13:16:10,976 - INFO - Generando gráfico: Ocurrencias vs Éxito por Patrón...


2025-04-06 13:16:11,201 - INFO - Gráfico guardado: data/images_analysis\pattern_counts_vs_success_comparison.png


In [35]:
# %%
logging.info("Generando gráfico (escala lineal): Ocurrencias vs Éxito por Patrón...")

if 'df_tasas_exito' not in locals() or not isinstance(df_tasas_exito, pd.DataFrame) or df_tasas_exito.empty:
    logging.warning("'df_tasas_exito' no disponible o vacío. No se genera gráfico comparativo.")
else:
    # Ordenar por tasa de éxito para mantener consistencia visual
    df_grafico = df_tasas_exito.sort_values(by='rate', ascending=False).copy()

    figura_comparativa_patrones = go.Figure()

    # Barra para Ocurrencias Totales
    figura_comparativa_patrones.add_trace(go.Bar(
        name='Ocurrencias Totales',
        x=df_grafico.index, # Patrón en eje X
        y=df_grafico['count'], # Conteo en eje Y
        text=df_grafico['count'].apply(lambda x: f'{x:,}' if x > 0 else ''),
        textposition='outside', # Texto fuera para barras grandes
        marker_color='lightskyblue',
        offsetgroup=1,
        customdata=df_grafico['count'],
        hovertemplate="<b>%{x}</b><br>Total Ocurrencias: %{customdata:,}<extra></extra>"
    ))

    # Barra para Ocurrencias con Éxito
    customdata_exito = df_grafico[['success_count', 'rate']].copy()
    customdata_exito['rate_str'] = customdata_exito['rate'].apply(lambda r: f"{r:.2f}%")

    figura_comparativa_patrones.add_trace(go.Bar(
        name='Ocurrencias con Éxito',
        x=df_grafico.index, # Patrón en eje X
        y=df_grafico['success_count'], # Conteo éxito en eje Y
        text=df_grafico['success_count'].apply(lambda x: f'{x:,}' if x > 0 else ''),
        textposition='auto', # Texto dentro o fuera según quepa
        marker_color='mediumseagreen',
        offsetgroup=1,
        base=0, # Asegurar que empiezan desde 0
        customdata=customdata_exito[['success_count', 'rate_str']],
        hovertemplate=("<b>%{x}</b><br>" +
                       "Éxito: %{customdata[0]:,}<br>" +
                       "Tasa Éxito: %{customdata[1]}" +
                       "<extra></extra>")
    ))

    titulo_grafico = 'Comparativa: Ocurrencias Totales vs. Éxito por Patrón (Escala Lineal)'
    figura_comparativa_patrones.update_layout(
        title=titulo_grafico,
        xaxis_title='Patrón',
        yaxis_title='Número de Ocurrencias', # Título eje Y - Escala Lineal
        barmode='group', # Barras agrupadas
        # yaxis_type="log", # <--- Línea eliminada para usar escala lineal
        # Mantener orden según tasa de éxito
        xaxis={'categoryorder':'array', 'categoryarray': df_grafico.index},
        legend_title_text="Tipo",
        height=500,
        width=1000,
        bargap=0.15, # Espacio entre barras del mismo grupo
        bargroupgap=0.1 # Espacio entre grupos de barras
    )
    figura_comparativa_patrones.show()

    try:
        directorio_salida = 'data/images_analysis'
        os.makedirs(directorio_salida, exist_ok=True)
        # Modificar nombre para reflejar el cambio a lineal
        nombre_archivo = "pattern_counts_vs_success_comparison_linear.png"
        ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
        # Guardar con dimensiones correctas
        figura_comparativa_patrones.write_image(ruta_imagen, width=1000, height=500, scale=1)
        logging.info(f"Gráfico guardado: {ruta_imagen}")
    except Exception as err:
        logging.error(f"Error al guardar gráfico comparativo de patrones: {err}")

2025-04-06 13:26:04,585 - INFO - Generando gráfico (escala lineal): Ocurrencias vs Éxito por Patrón...


2025-04-06 13:26:04,908 - INFO - Gráfico guardado: data/images_analysis\pattern_counts_vs_success_comparison_linear.png


In [27]:
logging.info("Calculando y graficando porcentaje de ocurrencia general por patrón...")

if 'df_tasas_exito' not in locals() or not isinstance(df_tasas_exito, pd.DataFrame) or df_tasas_exito.empty:
    logging.warning("'df_tasas_exito' no disponible o vacío.")
elif 'df_procesado' not in locals() or not isinstance(df_procesado, pd.DataFrame):
     logging.warning("'df_procesado' no disponible.")
else:
    total_filas = len(df_procesado)
    if total_filas == 0:
        logging.warning("'df_procesado' está vacío.")
    else:
        df_ocurrencia = df_tasas_exito[['count']].copy()
        df_ocurrencia['porcentaje_ocurrencia'] = (df_ocurrencia['count'] / total_filas) * 100
        df_ocurrencia = df_ocurrencia.sort_values(by='porcentaje_ocurrencia', ascending=False)

        print("\nPorcentaje Ocurrencia General por Patrón:")
        print(df_ocurrencia[['porcentaje_ocurrencia']].to_string(formatters={'porcentaje_ocurrencia': '{:,.2f}%'.format}))

        figura_ocurrencia = go.Figure()

        figura_ocurrencia.add_trace(go.Bar(
            x=df_ocurrencia.index,
            y=df_ocurrencia['porcentaje_ocurrencia'],
            name='Ocurrencia (%)',
            text=df_ocurrencia['porcentaje_ocurrencia'].apply(lambda x: f'{x:.2f}%' if x > 0.01 else ''), # No mostrar texto si es muy pequeño
            textposition='outside',
            marker_color='cornflowerblue',
            customdata=df_ocurrencia['count'],
            hovertemplate=(
                f"<b>Patrón: %{{x}}</b><br>" +
                f"Ocurrencia: %{{y:.3f}}%<br>" +
                f"Conteo: %{{customdata:,}}<br>" +
                f"(Total Filas: {total_filas:,})<extra></extra>"
            )
        ))

        titulo_grafico = 'Porcentaje de Ocurrencia General por Patrón'
        figura_ocurrencia.update_layout(
            title=titulo_grafico,
            xaxis_title='Patrón',
            yaxis_title='Ocurrencia (% del Total de Filas)',
            xaxis={'categoryorder':'total descending'},
            yaxis_ticksuffix="%",
            height=500,
            width=1000
        )
        figura_ocurrencia.show()

        try:
            directorio_salida = 'data/images_analysis'
            os.makedirs(directorio_salida, exist_ok=True)
            nombre_archivo = "pattern_overall_occurrence_percentage.png"
            ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
            figura_ocurrencia.write_image(ruta_imagen, scale=1)
            logging.info(f"Gráfico guardado: {ruta_imagen}")
        except Exception as err:
            logging.error(f"Error al guardar gráfico de ocurrencia: {err}")

2025-04-06 13:16:11,243 - INFO - Calculando y graficando porcentaje de ocurrencia general por patrón...



Porcentaje Ocurrencia General por Patrón:
                   porcentaje_ocurrencia
Doji                              10.83%
BearishEngulfing                   6.65%
BullishEngulfing                   6.37%
Sinusoidal                         2.35%
Hammer                             1.95%
HangingMan                         1.95%
ThreeWhiteSoldiers                 1.28%
ThreeBlackCrows                    1.25%
DarkCloudCover                     0.88%
PiercingLine                       0.80%
EveningStar                        0.54%
MorningStar                        0.50%


2025-04-06 13:16:11,418 - INFO - Gráfico guardado: data/images_analysis\pattern_overall_occurrence_percentage.png


In [28]:
logging.info("Generando gráfico: tasa de éxito por patrón...")

if 'df_tasas_exito' not in locals() or not isinstance(df_tasas_exito, pd.DataFrame) or df_tasas_exito.empty:
    logging.warning("'df_tasas_exito' no disponible o vacío.")
else:
    df_tasa_exito_grafico = df_tasas_exito[['rate', 'count', 'success_count']].copy()
    df_tasa_exito_grafico = df_tasa_exito_grafico.sort_values(by='rate', ascending=False)

    print("\nTasa de Éxito por Patrón:")
    print(df_tasa_exito_grafico[['rate']].to_string(formatters={'rate': '{:,.2f}%'.format}))

    figura_tasa_exito = go.Figure()

    hover_base = (f"<b>Patrón: %{{x}}</b><br>" +
                  f"Tasa Éxito (Subida≥{MIN_FUTURE_INCREASE_PERCENT*100:.1f}%): %{{y:.2f}}%<br>" +
                  f"Ocurrencias: %{{customdata[0]:,}}<br>" +
                  f"Éxitos: %{{customdata[1]:,}}<extra></extra>")
    hover_sinusoidal = (f"<b>Patrón: %{{x}}</b><br>" +
                       f"Tasa Éxito (Target=1): %{{y:.2f}}%<br>" +
                       f"Formas Detectadas: %{{customdata[0]:,}}<br>" +
                       f"Targets: %{{customdata[1]:,}}<extra></extra>")

    textos_hover = [
        hover_sinusoidal if idx == "Sinusoidal" else hover_base
        for idx in df_tasa_exito_grafico.index
    ]

    figura_tasa_exito.add_trace(go.Bar(
        x=df_tasa_exito_grafico.index,
        y=df_tasa_exito_grafico['rate'],
        name='Tasa Éxito (%)',
        text=df_tasa_exito_grafico['rate'].apply(lambda x: f'{x:.1f}%'),
        textposition='auto',
        marker_color='darkorange',
        customdata=df_tasa_exito_grafico[['count', 'success_count']],
        hovertemplate=textos_hover
    ))

    try:
        if 'df_procesado' in locals() and 'future_max_increase_capped' in df_procesado.columns:
            condicion_exito_general = (df_procesado['future_max_increase_capped'] >= MIN_FUTURE_INCREASE_PERCENT)
            tasa_exito_general = condicion_exito_general.mean() * 100
            total_subidas_minimas = condicion_exito_general.sum()
            figura_tasa_exito.add_hline(
                y=tasa_exito_general,
                line_dash="dot",
                line_color="grey",
                annotation_text=f"Tasa Base Subida≥{MIN_FUTURE_INCREASE_PERCENT*100:.1f}% ({tasa_exito_general:.2f}%, N={total_subidas_minimas:,})",
                annotation_position="bottom right"
            )
    except Exception as e:
        logging.warning(f"No se pudo añadir línea de tasa base: {e}")

    titulo_grafico = 'Tasa de Éxito por Patrón'
    figura_tasa_exito.update_layout(
        title=titulo_grafico,
        xaxis_title='Patrón',
        yaxis_title='Tasa de Éxito (%)',
        xaxis={'categoryorder':'total descending'},
        yaxis_ticksuffix="%",
        height=500,
        width=1000,
        hoverlabel=dict(bgcolor="white", font_size=12)
    )
    figura_tasa_exito.show()

    try:
        directorio_salida = 'data/images_analysis'
        os.makedirs(directorio_salida, exist_ok=True)
        nombre_archivo = "pattern_success_rate.png"
        ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
        figura_tasa_exito.write_image(ruta_imagen, scale=1)
        logging.info(f"Gráfico guardado: {ruta_imagen}")
    except Exception as err:
        logging.error(f"Error al guardar gráfico de tasa de éxito: {err}")

2025-04-06 13:16:11,455 - INFO - Generando gráfico: tasa de éxito por patrón...



Tasa de Éxito por Patrón:
                    rate
ThreeWhiteSoldiers 6.28%
BullishEngulfing   4.82%
Hammer             3.48%
HangingMan         3.48%
DarkCloudCover     3.01%
Doji               2.98%
PiercingLine       2.94%
MorningStar        2.75%
BearishEngulfing   2.58%
ThreeBlackCrows    2.38%
EveningStar        1.98%
Sinusoidal         1.68%


2025-04-06 13:16:11,718 - INFO - Gráfico guardado: data/images_analysis\pattern_success_rate.png


In [29]:
logging.info("Generando histograma: distribución de duración para target=1...")

columna_duracion = 'duration_minutes'
if 'target' not in df_procesado.columns or columna_duracion not in df_procesado.columns:
    logging.warning(f"Columnas 'target' o '{columna_duracion}' no encontradas.")
else:
    datos_target_1_duracion = df_procesado[
        (df_procesado['target'] == 1) & (df_procesado[columna_duracion] > 0)
    ].copy()

    if datos_target_1_duracion.empty:
        logging.warning("No hay datos con target=1 y duración válida para generar histograma.")
    else:
        duraciones = datos_target_1_duracion[columna_duracion]

        figura_duracion = go.Figure()

        figura_duracion.add_trace(go.Histogram(
            x=duraciones,
            name='Duración',
            marker_color='mediumaquamarine',
            xbins=dict(start=duraciones.min() - 0.5, end=duraciones.max() + 0.5, size=1),
            hovertemplate='Duración: %{x} min<br>Frecuencia: %{y:,}<extra></extra>'
        ))

        media_duracion = duraciones.mean()
        mediana_duracion = duraciones.median()

        figura_duracion.add_vline(x=media_duracion, line_dash="dash", line_color="salmon",
                                  annotation_text=f"Media={media_duracion:.1f}", annotation_position="top left")
        figura_duracion.add_vline(x=mediana_duracion, line_dash="dot", line_color="lightcoral",
                                  annotation_text=f"Mediana={mediana_duracion:.0f}", annotation_position="top right")

        titulo_grafico = 'Distribución de Duración (Pico-Valle) para Target=1'
        figura_duracion.update_layout(
            title=titulo_grafico,
            xaxis_title='Duración (Minutos)',
            yaxis_title='Frecuencia (Ocurrencias)',
            bargap=0.1,
            height=500,
            width=1000
        )

        # Asegurar que el rango del eje X cubra MIN y MAX definidos si hay datos
        rango_x_min = min(MIN_DURATION - 1, duraciones.min() - 1)
        rango_x_max = max(MAX_DURATION + 1, duraciones.max() + 1)
        figura_duracion.update_xaxes(range=[rango_x_min, rango_x_max])

        figura_duracion.show()

        try:
            directorio_salida = 'data/images_analysis'
            os.makedirs(directorio_salida, exist_ok=True)
            nombre_archivo = "target1_duration_histogram.png"
            ruta_imagen = os.path.join(directorio_salida, nombre_archivo)
            figura_duracion.write_image(ruta_imagen, scale=1)
            logging.info(f"Gráfico guardado: {ruta_imagen}")
        except Exception as err:
            logging.error(f"Error al guardar histograma de duración: {err}")

2025-04-06 13:16:11,809 - INFO - Generando histograma: distribución de duración para target=1...


2025-04-06 13:16:11,971 - INFO - Gráfico guardado: data/images_analysis\target1_duration_histogram.png


In [30]:
df_procesado.sample(5)

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_lag_0,volume_lag_0,open_lag_0,high_lag_0,low_lag_0,quote_asset_volume_lag_0,number_of_trades_lag_0,taker_buy_base_asset_volume_lag_0,taker_buy_quote_asset_volume_lag_0,close_lag_1,volume_lag_1,open_lag_1,high_lag_1,low_lag_1,quote_asset_volume_lag_1,number_of_trades_lag_1,taker_buy_base_asset_volume_lag_1,taker_buy_quote_asset_volume_lag_1,close_lag_2,volume_lag_2,open_lag_2,high_lag_2,low_lag_2,quote_asset_volume_lag_2,number_of_trades_lag_2,taker_buy_base_asset_volume_lag_2,taker_buy_quote_asset_volume_lag_2,close_lag_3,volume_lag_3,open_lag_3,high_lag_3,low_lag_3,quote_asset_volume_lag_3,number_of_trades_lag_3,taker_buy_base_asset_volume_lag_3,taker_buy_quote_asset_volume_lag_3,close_lag_4,volume_lag_4,open_lag_4,high_lag_4,low_lag_4,quote_asset_volume_lag_4,number_of_trades_lag_4,taker_buy_base_asset_volume_lag_4,taker_buy_quote_asset_volume_lag_4,close_lag_5,volume_lag_5,open_lag_5,high_lag_5,low_lag_5,quote_asset_volume_lag_5,number_of_trades_lag_5,taker_buy_base_asset_volume_lag_5,taker_buy_quote_asset_volume_lag_5,close_lag_6,volume_lag_6,open_lag_6,high_lag_6,low_lag_6,quote_asset_volume_lag_6,number_of_trades_lag_6,taker_buy_base_asset_volume_lag_6,taker_buy_quote_asset_volume_lag_6,close_lag_7,volume_lag_7,open_lag_7,high_lag_7,low_lag_7,quote_asset_volume_lag_7,number_of_trades_lag_7,taker_buy_base_asset_volume_lag_7,taker_buy_quote_asset_volume_lag_7,close_lag_8,volume_lag_8,open_lag_8,high_lag_8,low_lag_8,quote_asset_volume_lag_8,number_of_trades_lag_8,taker_buy_base_asset_volume_lag_8,taker_buy_quote_asset_volume_lag_8,close_lag_9,volume_lag_9,open_lag_9,high_lag_9,low_lag_9,quote_asset_volume_lag_9,number_of_trades_lag_9,taker_buy_base_asset_volume_lag_9,taker_buy_quote_asset_volume_lag_9,close_lag_10,volume_lag_10,open_lag_10,high_lag_10,low_lag_10,quote_asset_volume_lag_10,number_of_trades_lag_10,taker_buy_base_asset_volume_lag_10,taker_buy_quote_asset_volume_lag_10,close_lag_11,volume_lag_11,open_lag_11,high_lag_11,low_lag_11,quote_asset_volume_lag_11,number_of_trades_lag_11,taker_buy_base_asset_volume_lag_11,taker_buy_quote_asset_volume_lag_11,close_lag_12,volume_lag_12,open_lag_12,high_lag_12,low_lag_12,quote_asset_volume_lag_12,number_of_trades_lag_12,taker_buy_base_asset_volume_lag_12,taker_buy_quote_asset_volume_lag_12,close_lag_13,volume_lag_13,open_lag_13,high_lag_13,low_lag_13,quote_asset_volume_lag_13,number_of_trades_lag_13,taker_buy_base_asset_volume_lag_13,taker_buy_quote_asset_volume_lag_13,close_lag_14,volume_lag_14,open_lag_14,high_lag_14,low_lag_14,quote_asset_volume_lag_14,number_of_trades_lag_14,taker_buy_base_asset_volume_lag_14,taker_buy_quote_asset_volume_lag_14,close_lag_15,volume_lag_15,open_lag_15,high_lag_15,low_lag_15,quote_asset_volume_lag_15,number_of_trades_lag_15,taker_buy_base_asset_volume_lag_15,taker_buy_quote_asset_volume_lag_15,close_lag_16,volume_lag_16,open_lag_16,high_lag_16,low_lag_16,quote_asset_volume_lag_16,number_of_trades_lag_16,taker_buy_base_asset_volume_lag_16,taker_buy_quote_asset_volume_lag_16,close_lag_17,volume_lag_17,open_lag_17,high_lag_17,low_lag_17,quote_asset_volume_lag_17,number_of_trades_lag_17,taker_buy_base_asset_volume_lag_17,taker_buy_quote_asset_volume_lag_17,close_lag_18,volume_lag_18,open_lag_18,high_lag_18,low_lag_18,quote_asset_volume_lag_18,number_of_trades_lag_18,taker_buy_base_asset_volume_lag_18,taker_buy_quote_asset_volume_lag_18,close_lag_19,volume_lag_19,open_lag_19,high_lag_19,low_lag_19,quote_asset_volume_lag_19,number_of_trades_lag_19,taker_buy_base_asset_volume_lag_19,taker_buy_quote_asset_volume_lag_19,close_lag_20,volume_lag_20,open_lag_20,high_lag_20,low_lag_20,quote_asset_volume_lag_20,number_of_trades_lag_20,taker_buy_base_asset_volume_lag_20,taker_buy_quote_asset_volume_lag_20,close_lag_21,volume_lag_21,open_lag_21,high_lag_21,low_lag_21,quote_asset_volume_lag_21,number_of_trades_lag_